In [ ]:
# Before you start:

# The data that support the findings of this study are openly available in NIOZ Dataverse at http://doi.org/[doi], reference number [reference number].
# To run the code below, you should download the data into the raw data directory in the "wd organization" cell
# For reference of this asset, please refer to the file "CITATION.cff"

In [ ]:
# This notebook is used for analyzing time series of primary production collected at Jetty monitoring plattform:
#install.packages("plotly")
#install.packages("pracma")
library(mgcv)
library(ggplot2)
library(lubridate)
library(plotly)
library(dplyr)
library(pracma)

In [ ]:
# wd organization (always run first)

#================================================================================
#     START OF USER DEFINED INPUT
#================================================================================
# wd_path <- readClipboard()
# setwd(wd_path)
wd_path <- "/home/jovyan/Virtual Labs/Waddenzee proto DT/Git public/Jetty_PPts/"


# Check if the directories exists
rawdata_dir = paste(wd_path,"/rawdata/",sep="")
processed_data_dir = paste(wd_path,"/processed_data/",sep="")
PI_output_dir = paste(wd_path,"/PI_output/",sep="")
PP_cal_dir = paste(wd_path,"/PP_cal/",sep="")
graph_dir = paste(wd_path,"/graphs/",sep="")

if (!dir.exists(wd_path)) {
  # Create the directory if it doesn't exist
  dir.create(wd_path, recursive = TRUE)
}
if (!dir.exists(rawdata_dir)) {
  # Create the directory if it doesn't exist
  dir.create(rawdata_dir, recursive = TRUE)
}
if (!dir.exists(processed_data_dir)) {
  # Create the directory if it doesn't exist
  dir.create(processed_data_dir, recursive = TRUE)
}
if (!dir.exists(PI_output_dir)) {
  # Create the directory if it doesn't exist
  dir.create(PI_output_dir, recursive = TRUE)
}
if (!dir.exists(PP_cal_dir)) {
  # Create the directory if it doesn't exist
  dir.create(PP_cal_dir, recursive = TRUE)
}
if (!dir.exists(graph_dir)) {
  # Create the directory if it doesn't exist
  dir.create(graph_dir, recursive = TRUE)
}



In [ ]:
# 1. process raw data
## 1.1 process PI data

### refer to manual, pdf file: "Primary production calculation manual_V1.pdf"

InputData_DPM = read.csv(file=paste(wd_path,"/rawdata/","DPM.csv",sep=""), fileEncoding = "UTF-8", na.strings=c("", "NA"), header=TRUE)
InputData_DPM$HW <- trimws(InputData_DPM$HW)
InputData_DPM$HW <- sub(" ", "", InputData_DPM$HW, fixed = TRUE) # remove white space in between

Data_DPM = InputData_DPM[,c("HW","sample","DPM1")]
names(Data_DPM) = c("HW","sample","DPM_sample")
Data_DPM$DPM_sample = Data_DPM$DPM_sample%>%as.character()%>%as.numeric()

InputData_TTDPMC = read.csv(file=paste(wd_path,"/rawdata/","RUW.csv",sep=""), fileEncoding = "UTF-8", na.strings=c("", "NA"), header=TRUE)
InputData_TTDPMC$HT <- trimws(InputData_TTDPMC$HT)
InputData_TTDPMC$HT <- sub(" ", "", InputData_TTDPMC$HT, fixed = TRUE) # remove white space in between

# Data_TTDPMC = InputData_TTDPMC[,c("HT","Sampling.Date","Sampling.Time","Chla_ug_per_l","DPM.1000","Temp..rel..DIC..mg.l.1.","Jetty.insitu_T","incubation_T","inc..duur..h.")]
Data_TTDPMC = InputData_TTDPMC[,c("HT","Sampling_Date","Sampling_Time","SPMCHLA_PLUS_µg_per_l","SDD","DPM_added_C","Temp_rel_DIC","Jetty_insitu_T","incubation_T","inc_duur","Licht_1","Licht_4","Licht_5","Licht_9","Licht_10")]
names(Data_TTDPMC) = c("HW","Date","Time","Chla_ug_per_l","Secchi_D","DPM_added","DIC","T_insitu","T_incubation","Incubation_duration_h","Licht_1","Licht_4","Licht_5","Licht_9","Licht_10")
Data_TTDPMC$T_insitu <- Data_TTDPMC$T_insitu%>%as.character()%>%as.numeric()
Data_TTDPMC$DIC = ((Data_TTDPMC$T_insitu^2)*0.0058)+(Data_TTDPMC$T_insitu*(-0.3169))+28.601
Data_TTDPMC$Date <- Data_TTDPMC$Date%>%dmy

InputData_DPMD = Data_DPM[which(Data_DPM$sample%in%c("DA","DB")),]
Data_DPM_noD = Data_DPM[-which(Data_DPM$sample%in%c("DA","DB","CAS","C","C1","C2","C3")),]

InputData_DPMD$DPM_sample <- InputData_DPMD$DPM_sample%>%as.character()%>%as.numeric()

Data_DPMD = InputData_DPMD %>% group_by(HW) %>% summarise(DPM_dark = mean(DPM_sample, na.rm=TRUE))

Data_TTDPM_C_D = merge(Data_TTDPMC,Data_DPMD,by="HW",all=TRUE)

#to calculate C fixation rate data from both df are needed, but these are of unequal length (480 en 24 resp).
#first merge dfs into one

Data_T <- merge(Data_TTDPM_C_D, Data_DPM_noD, by="HW",all = T)
Data_T$T_incubation <- Data_T$T_incubation%>%as.character()%>%as.numeric()
Data_T$T_insitu <- Data_T$T_insitu%>%as.character()%>%as.numeric()
Data_T$DPM_added <- Data_T$DPM_added%>%as.character()%>%as.numeric()
# Data_T$Incubation_duration_h <- Data_T$Incubation_duration_h%>%hms 

Data_T$Temp_factor = (exp(0.0693*(Data_T$T_insitu-Data_T$T_incubation)))
#calculate C fixation rate (mg C L-1 h-1) by
#DPM sample-DPM dark x 1.05 discrimination factor 14C/13C x [DIC] x temperature correction factor/DPM controle x duration of inc
Data_T$carbon <- ((Data_T$DPM_sample-Data_T$DPM_dark) * 1.05 *(Data_T$DIC) * Data_T$Temp_factor) / ((Data_T$DPM_added) * (Data_T$Incubation_duration_h)) # unit: mgC L-1 h-1
range(Data_T$carbon,na.rm = T) # unit: mgC L-1 h-1

Data_T$carbon_per_chla <- Data_T$carbon/(Data_T$Chla_ug_per_l*1e-3) # unit: mg C (mg Chla)-1 h-1
Data_T <- Data_T[!is.na(Data_T$sample),]

# now add a column with the light received per sample in the incubator to 'Data'

Data_Light = Data_T[,c("HW","Date","Time","Licht_1","Licht_4","Licht_5","Licht_9","Licht_10")]
names(Data_Light)[1:3] = c("HW","Date","Time")
#Data_Light <- na.omit(Data_Light)

library(zoo)
Data_Light <- Data_Light %>%
  mutate(across(starts_with("Licht"), as.numeric))

Data_Light <- Data_Light %>%
  mutate(across(starts_with("Licht"), ~replace(., .==0, NA)))

Light_data_interpolated <- Data_Light %>%
  mutate(across(starts_with("Licht"), ~ na.approx(., na.rm = FALSE)))

Light_data_filled <- Light_data_interpolated %>%
  mutate(across(starts_with("Licht"), ~ na.locf(., fromLast = TRUE, na.rm = FALSE))) %>%
  mutate(across(starts_with("Licht"), ~ na.locf(., na.rm = FALSE)))

Light_data_filled$Licht_2 = Light_data_filled$Licht_1 + (Light_data_filled$Licht_4-Light_data_filled$Licht_1)/3
Light_data_filled$Licht_3 = Light_data_filled$Licht_1 + 2*(Light_data_filled$Licht_4-Light_data_filled$Licht_1)/3
Light_data_filled$Licht_6 = Light_data_filled$Licht_5 + (Light_data_filled$Licht_9-Light_data_filled$Licht_5)/4
Light_data_filled$Licht_7 = Light_data_filled$Licht_5 + 2*(Light_data_filled$Licht_9-Light_data_filled$Licht_5)/4
Light_data_filled$Licht_8 = Light_data_filled$Licht_5 + 3*(Light_data_filled$Licht_9-Light_data_filled$Licht_5)/4

Data_T$Incubation_light <- NA
for(i in 1:nrow(Data_T)){
  light_col <- paste0("Licht_", as.numeric(gsub("[A-B]", "", Data_T$sample[i])))
  Data_T$Incubation_light[i] = Light_data_filled[i,light_col]  
}

Data_T<- Data_T[,-which(names(Data_T)%in%c("Licht_1","Licht_4","Licht_5","Licht_9","Licht_10"))]
#-------------------------------------------------------------------------------------------------------------
# Save finished df to dir_data
# Data file can be used as input in PE_curve_fit
# ------------------------------------------------------------------------------------------------------------



PI_input_2013_2023 = Data_T[,c("HW", "Incubation_light", "carbon", "carbon_per_chla")]
PI_2012 = read.csv(paste(rawdata_dir,"PI_2012.csv", sep="")) # Data from Jacobs et al., (2020)
names(PI_2012) = names(PI_input_2013_2023)
PI_input = rbind(PI_2012, PI_input_2013_2023)

PI_input$carbon_per_chla[which(PI_input$carbon_per_chla<0)]<- NA

# plot(Data_T$Incubation_light[which(Data_T$HW=="HW1501")],
#      Data_T$carbon_per_chla[which(Data_T$HW=="HW1501")],ylim=c(0,2))

# write.csv(PI_input, file = paste(wd_path,"processed_data/","PI_Input.csv",sep=""),row.names = F)

In [ ]:
## 1.2 Process Jetty HW series

library(ggplot2)   # Plotting
library(mgcv)      # Generalized Additive Models (GAM)
library(dplyr)     # Data wrangling
library(lubridate) # Date handling

# Set the path to your folder containing the CSV files
folder_path <- paste(rawdata_dir, "HWseries/", sep="") # Replace with your actual folder path

file_list <- list.files(path = folder_path, 
                       pattern = "HWseries_\\d{4}\\.csv$",
                       full.names = TRUE)

# Initialize an empty list to store the data frames
combined_data <- NULL

# Loop through each year and read the corresponding CSV file
for (file_path in file_list) {

  # Check if the file exists before trying to read it
  if (file.exists(file_path)) {
    # Read the CSV file, skipping the first 3 rows
    data <- read.csv(file_path, skip = 3, header = TRUE)
    
    end_col = which(names(data)=="Chl")
    # Add the data to our list
    combined_data <- rbind(combined_data, data[-1,1:end_col])
    units = data[1,]
  } else {
    warning(paste("File not found:", file_path))
  }
}

head(combined_data)

# View the structure of the combined data
str(combined_data)

names(combined_data)[2] <- "timestamp"
combined_data$timestamp[which(combined_data$SampleID%in%paste0("HW",c(1201:1240, 1301:1340, 1401:1440, 1501:1540, 1601:1640, 2301:2340, 1601:1640)))] <- dmy_hm(combined_data$timestamp[which(combined_data$SampleID%in%paste0("HW",c(1201:1240, 1301:1340, 1401:1440, 1501:1540, 1601:1640, 2301:2340, 1601:1640)))])%>%as.character
combined_data$timestamp <- ymd_hms(combined_data$timestamp)

cols_to_convert <- 3:ncol(combined_data)
combined_data[cols_to_convert] <- lapply(combined_data[cols_to_convert], function(x){
  # first remove any commas (common in numeric data)
  x_clean <- gsub(",", "", x)
  # Then convert to numeric
  as.numeric(x_clean)
})

str(combined_data)

# Jetty secci to Kd, pars see Jacobs et al. 2020
# 1.476/Secci.SSD + 0.3541
# max secchi depth between 2010-2017 (based on "PrimProd_cfPhillippart_etal_2007.xlsx"): 3.24
secci_par_c = 1.476
secci_par_d = 0.3541
max_secci = 3.24
combined_data$Kd_Secci <- secci_par_c/combined_data$SecchiDepth + secci_par_d

# <- ifelse(PI_input$Secchi_D%in%c(">3.0",">3"), (secci_par_c/max_secci + secci_par_d), 
#                          (secci_par_c/(PI_input$Secchi_D%>%as.character()%>%as.numeric()) + secci_par_d))


# Or based on SPM (Loebl et al., 2009) 
# Kd = 0.188 + 0.024*SPM 

combined_data$Kd_SPM <- combined_data$TSM*0.024 + 0.188
# plot(combined_data$timestap,combined_data$Kd_SPM)
# write.csv(combined_data, file = paste(wd_path,"/processed_data/","Jetty_HWseries.csv",sep=""),row.names = F)

In [ ]:
# 2. Fit PI curves
## Eilers & Peeters model 

PI_input = read.csv(paste(processed_data_dir,"PI_Input.csv", sep=""))

# adapted from EMS_Dollar 
sink(paste(PI_output_dir,"output.csv",sep=""))
cat("HWcode,a_fit,b_fit,c_fit,a_std_err,b_std_err,c_std_err,a_low,a_upp,b_low,b_upp,c_low,c_upp,residual_sum_of_squares,is_converged?,stop_message,number_of_iterations,degrees_of_freedom\n")
sink()

# Define the Michaelis-Menten model function
mm_model <- function(I, Pmax, K) {
  Pmax * I / (K + I)
}

# function to fit to data
fit_function <- function(x, a, b, c)
{
  return (x/(a*x^2 + b*x + c))
}

fit <- function(x,y,HWcode)
{
  # Make a best initial guess of parameters a, b, and c of fit_function
  # 1/c is equal to slope at x=0. Hence, c is estimate from the data at x=0 and the first x-value after that.
  x1 <- sort(x[x != 0])[1]
  y1 <- y[x != 0][order(x[x != 0])][1]
  c_guess <- x1/y1
  if (c_guess < 0) c_guess <- -c_guess
  # b = 1/Pm - 2*c/Im
  Pm <- max(y)
  Im <- x[which(y == Pm)][1]
  b_guess <- (1/Pm) - (c_guess*2/Im)
  a_guess <- c_guess/(Im^2)
  
  
  
  # Michaelis-Menten model intial fitting for outlier removal
  # Pmax_guess = max(y, na.rm = T)
  # K_guess = median(x, na.rm = T)
  # model3 <- NULL
  # model3 <- nls(y ~ mm_model(x, Pmax,K), start = list(Pmax = Pmax_guess, K=K_guess))
  # residuals <- abs(resid(model3))
  # threshold <- 2.5 * sd(residuals)
  # outliers <- residuals > threshold
  
  # # # Iterative outlier removal
  # model0 <- NULL
  # model0 <- nls(y ~ fit_function(x, a, b, c), start = list(a = a_guess, b = b_guess, c = c_guess),
  #                 trace = TRUE, control = list(warnOnly = TRUE))
  # residuals <- abs(resid(model0))
  # threshold <- 2 * sd(residuals)
  # outliers <- residuals > threshold
  
  # x_corr <- x[!outliers]
  # y_corr <- y[!outliers]
  x_corr <- x
  y_corr <- y
  
  model <- NULL
  model <- nls(y_corr ~ fit_function(x_corr, a, b, c), start = list(a = a_guess, b = b_guess, c = c_guess),
               trace = TRUE, control = list(warnOnly = TRUE),algorithm = "port")
  #
  
  # model <- NULL
  # model <- nls(y ~ fit_function(x, a, b, c), start = list(a = a_guess, b = b_guess, c = c_guess),
  #              trace = TRUE, control = list(warnOnly = TRUE))
  # 
  
  pars <- summary(model)$parameters[1:3, 1]
  pars_unc <- summary(model)$parameters[1:3, 2]
  
  filename <- HWcode
  
  sink(paste(PI_output_dir,"output.csv",sep=""), append = T)
  cat(HWcode)
  cat(",")
  cat(pars, sep =",")
  cat(",")
  cat(pars_unc, sep =",")
  sink()
  
  # x2 <- seq(0, 1800, len = 50)
  x2 <- seq(0, max(x), len = 50)
  myenv <- new.env()
  assign("a1", pars[1], envir = myenv)
  assign("b1", pars[2], envir = myenv)
  assign("c1", pars[3], envir = myenv)
  assign("x1", x2, envir = myenv)
  ff <- numericDeriv(quote(fit_function(x1, a1, b1, c1)), c("a1", "b1", "c1"), myenv)
  se.fit <- NULL
  se.fit <- sqrt(apply(attr(ff, "gradient"), 1, function(x) sum(vcov(model)*outer(x, x))))
  sumofsquares <- sum(summary(model)$residuals^2)
  degreesoffreedom <- summary(model)$df[2]
  # change confidence below for different confidence bands
  confidence <- 0.95
  t_values <- -qt((1-confidence)/2, degreesoffreedom)
  t_values <- c(-t_values, 0, t_values)
  
  sink(paste(PI_output_dir,"output.csv",sep=""), append = T)
  for (i in 1:3)
  {
    cat(",")
    cat(pars[i] + t_values[-2]*pars_unc[i], sep = ",")
  }
  cat(",")
  cat(sum(summary(model)$residuals^2))
  cat(",")
  cat(model$convInfo$isConv)
  cat(",")
  cat(model$convInfo$stopMessage)
  cat(",")
  cat(summary(model)$convInfo$finIter)
  cat(",")
  cat(degreesoffreedom)
  cat("\n")
  sink()
  
  pred <- NULL
  pred <- predict(model, list(x_corr = x2)) + outer(se.fit, t_values)
  
  png(paste(PI_output_dir, filename, ".png",sep=""))
  matplot(x2, pred, type="l", col=c(2,1,2), lty=c(2,1,2), 
          xlab = "PAR uE/m2/s", ylab = "Production mgC/mg Chla/h", 
          ylim=range(c(y, pred)), main=filename)
  points(x, y,col="darkred",pch=19)
  points(x_corr, y_corr,col="darkgreen",pch=19)
  
  dev.off()
  summary(model)
  
  #  print(paste("initial slope:", 1/pars[3]))
  slope_std_err <- sqrt((pars_unc[3]/pars[3])^2)/pars[3]
  #  print(paste("std err:", slope_std_err))
  slope_conf <- (1/pars[3]) + t_values*slope_std_err
  #  print(paste("95% lower and upper conf limit:", slope_conf[1], slope_conf[3]))
  
  c_a_ratio = pars[3]/pars[1]
  ifelse(c_a_ratio > 0, max_ETR_at <- sqrt(c_a_ratio), max_ETR_at<-NA)
  
  myenv <- new.env()
  assign("a1", pars[1], envir = myenv)
  assign("b1", pars[2], envir = myenv)
  assign("c1", pars[3], envir = myenv)
  assign("x1", max_ETR_at, envir = myenv)
  # ff <- NULL
  # ff <- numericDeriv(quote(fit_function(x1, a1, b1, c1)), c("a1", "b1", "c1"), myenv)
  # se.fit <- NULL
  # se.fit <- sqrt(apply(attr(ff, "gradient"), 1, function(x) sum(vcov(model)*outer(x, x))))
  # #  print(paste("ETRmax:", 1/(pars[2]+2*sqrt(pars[1]*pars[3]))))
  # #  print(paste("std err:", se.fit[1]))
  # max_conf <- predict(model, list(x = max_ETR_at)) + outer(se.fit, t_values)
  #  print(paste("95% lower and upper conf limit:", max_conf[1], max_conf[3]))
  sink(paste(PI_output_dir, filename, ".csv", sep=""))
  cat("x_fit,low_fit,med_fit,upp_fit,measured_x,measured_y,")
  cat(filename)
  cat("\n")
  temp <- cbind(x2, pred, c(x, rep(NA, length(x2) - length(x))), c(y, rep(NA, length(x2) - length(x))))
  for (i in 1:nrow(temp))
  {
    cat(temp[i,], sep = ",")
    cat("\n")
  }
  sink()
  
  return(list(HWcode,pars=pars))
}

# load processed incubation data and fit PI curve
# load processed incubation data and fit PI curve
remove_outliers <- function(x, y) {
  Q1 <- quantile(y, 0.25,na.rm = T)
  Q3 <- quantile(y, 0.75,na.rm = T)
  IQR <- Q3 - Q1
  lower_bound <- Q1 - 1.5 * IQR
  upper_bound <- Q3 + 1.5 * IQR
  non_outliers <- y >= lower_bound & y <= upper_bound
  list(x = x[non_outliers], y = y[non_outliers])
}

# PI_input <- na.omit(PI_input[,c("HW","Date","Time","datum","Incubation_light","carbon_per_chla","carbon")])
PI_input$HW <- as.factor(PI_input$HW)

for (HWcode in levels(PI_input$HW)) {
  data = PI_input[which(PI_input$HW==HWcode),]
  
  x = data$Incubation_light
  y = data$carbon_per_chla
  
  if(any(is.na(y))==F){
  # Remove outliers
  cleaned_data <- remove_outliers(x, y)
  x_clean <- cleaned_data$x
  y_clean <- cleaned_data$y
  
  fit(x_clean, y_clean,HWcode)}
}

# check HW2006 over high PP
# plot(data$Incubation_light, data$carbon)

PI_output <- read.csv(paste(PI_output_dir,"output.csv", sep=""))

# remove problematic PI curves, determined based on the comments in the lab notebook , in combination with visual inspection of PI curves, and how outliers are determined in Jacobs et al., 2020
# problem_PI_samples <- c("HW1629","HW1701","HW1708","HW1724")
problem_PI_samples <- c("HW1209","HW1210","HW1212", "HW1318", "HW1414", "HW1416", "HW1417","HW1724", "HW1728", "HW1911", "HW1913", "HW1938", "HW2020", "HW2024", "HW2031", "HW2103", "HW2121", "HW2217", "HW2231", "HW2233")

# HW1209 has a extremly high optimum light of 16000
# "HW1228" (30 Pmax); "HW1312" (30); "HW1327" keep increasing;
# PI_output$I_opt = sqrt(PI_output$c_fit/PI_output$a_fit)

PI_output_v2 <- PI_output[-which(PI_output$HWcode%in%problem_PI_samples),]

Jetty_df = read.csv(file=paste(wd_path,"/processed_data/","Jetty_HWseries.csv",sep=""), fileEncoding = "UTF-8", na.strings=c("", "NA"), header=TRUE)
Jetty_df$timestamp <- Jetty_df$timestamp%>%ymd_hms()

PI_output_v2$datum <- NA
for (i in 1:nrow(PI_output_v2)) {
  PI_output_v2$datum[i] = Jetty_df$timestamp[which(Jetty_df$SampleID==PI_output_v2$HWcode[i])]%>%as.character()
}

PI_output_v2$datum <- ymd_hms(PI_output_v2$datum)
# write.csv(PI_output_v2,file = paste(PI_output_dir,"output_v2.csv",sep=""),row.names = F)


In [ ]:
# 3. PP integration
## 3.1 EP PI model 2012-2023
require(pracma)

Jetty_df = read.csv(file=paste(wd_path,"/processed_data/","Jetty_HWseries.csv",sep=""), fileEncoding = "UTF-8", na.strings=c("", "NA"), header=TRUE)
Jetty_df$timestamp <- Jetty_df$timestamp%>%ymd_hms()

PI_output_v2 <- read.csv(paste(PI_output_dir,"output_v2.csv", sep=""))

cor(PI_output_v2[,c("a_fit","b_fit","c_fit")]) # results showed high intercorrelation between a, b,c. not in line with Ems-Dollard data report, in which they stated that c is less corrected with a or b.

PI_output_v2$datum <- ymd_hms(PI_output_v2$datum)

str(PI_output_v2$datum)
# plot(PI_output_v2$datum, PI_output_v2$a_fit)


cleaned_PI <- PI_output_v2%>%na.omit

# perform rectangular interpolation to derive daily values of Eilers-Peeters a, b, and c

daily_date <- seq.Date(from = ymd("2012-01-01"), to = ymd("2022-12-31"), by = "day")

days <- yday(daily_date)

# Function to perform rectangular interpolation for one year
days_in_year <- function(year) {
  start_date <- ymd(paste0(year, "-01-01"))
  end_date <- ymd(paste0(year, "-12-31"))
  return(as.integer(end_date - start_date + 1))
}

rectangular_interpolation <- function(year,days, values) {
  daily_values <- numeric(days_in_year(year))
  
  # Fill in the initial period with the first measurement
  daily_values[1:floor(days[1])] <- values[1]
  
  # Interpolate between measurements
  for (i in 1:(length(values) - 1)) {
    start_day <- floor(days[i])
    end_day <- floor(days[i + 1])
    mid_point <- floor((start_day + end_day) / 2)
    
    # Fill in the first half with the current measurement
    daily_values[start_day:mid_point] <- values[i]
    
    # Fill in the second half with the next measurement
    daily_values[(mid_point + 1):end_day] <- values[i + 1]
  }
  
  # Fill in the last period with the last measurement
  daily_values[floor(days[length(values)]):365] <- values[length(values)]
  
  return(daily_values)
}

# Apply the interpolation for each PI variable and each year
years <- cleaned_PI$datum%>%year()%>%unique
daily_PIdata <- data.frame()
for (year in years) {
  year_data <- cleaned_PI[cleaned_PI$datum%>%year == year, ]
  
  col_names <- c("a_fit","b_fit","c_fit","a_std_err","b_std_err","c_std_err","a_low","a_upp","b_low","b_upp","c_low","c_upp","residual_sum_of_squares")
  
  daily_values_list <- lapply(col_names, function(var) {
    rectangular_interpolation(year = year,year_data$datum%>%yday, year_data[[var]])
  })
  
  year_daily_data <- data.frame(
    Year = year,
    Day = 1:days_in_year(year),
    Date = paste(year,"-01-01")%>%ymd()+(1:days_in_year(year))
  )
  
  for (i in seq_along(daily_values_list)) {
    year_daily_data[[col_names[i]]] <- daily_values_list[[i]]
  }
  
  daily_PIdata <- rbind(daily_PIdata, year_daily_data)
}
# write.csv(daily_PIdata,file = paste(wd_path,"/processed_data/","daily_PIdata.csv",sep=""),row.names = F)


daily_df <- data.frame()

for (year in years) {
  year_data <- Jetty_df[Jetty_df$timestamp%>%year == year, ]
  
  col_names <- c("Chl","Kd_Secci")
  
  daily_values_list <- lapply(col_names, function(var) {
    rectangular_interpolation(year = year,year_data$timestamp%>%yday, year_data[[var]])
  })
  
  year_daily_data <- data.frame(
    Year = year,
    Day = 1:days_in_year(year),
    Date = paste(year,"-01-01")%>%ymd()+(1:days_in_year(year))
  )
  
  for (i in seq_along(daily_values_list)) {
    year_daily_data[[col_names[i]]] <- daily_values_list[[i]]
  }
  
  daily_df <- rbind(daily_df, year_daily_data)
}

plot(daily_df$Chl[which(daily_df$Year==2018)],type="l", ylab="Chla[ug/L]")
points(Jetty_df$timestamp[which(Jetty_df$timestamp%>%year==2018)]%>%yday,Jetty_df$Chl[which(Jetty_df$timestamp%>%year==2018)],pch=19,col="darkgreen")

plot(daily_df$Kd_Secci[which(daily_df$Year==2018)],type="l", ylab="Kd Secchi")
points(Jetty_df$timestamp[which(Jetty_df$timestamp%>%year==2018)]%>%yday,Jetty_df$Kd_Secci[which(Jetty_df$timestamp%>%year==2018)],pch=19,col="darkgreen")

hr_KNMI_filepath_2011_2020 <- paste(wd_path,"/rawdata/","uurgeg_235_2011-2020.txt",sep="")
hr_KNMI_filepath_2021_2030 <- paste(wd_path,"/rawdata/","uurgeg_235_2021-2030.txt",sep="")

hr_KNMI_deKooy_2011_20 <- read.table(hr_KNMI_filepath_2011_2020,header=TRUE, sep=",")
hr_KNMI_deKooy_2011_20$PAR = hr_KNMI_deKooy_2011_20$Q*1e4/3600*2.515 # Conversion from KNMI (W m-2) to PAR (uE m-2 s-1), reference Brinkman et al., 2015, EMS-Dollard data report
hr_KNMI_deKooy_2011_20$Datum <- paste(hr_KNMI_deKooy_2011_20$YYYYMMDD, hr_KNMI_deKooy_2011_20$HH,sep=" ")%>%ymd_h
# plot(hr_KNMI_deKooy_2011_20$Datum[1:365], hr_KNMI_deKooy_2011_20$PAR[1:365])

###
hr_KNMI_deKooy_2021_30 <- read.table(hr_KNMI_filepath_2021_2030, header=TRUE, sep=",")
hr_KNMI_deKooy_2021_30$PAR = hr_KNMI_deKooy_2021_30$Q*1e4/3600*2.515 # Conversion from KNMI (W m-2) to PAR (uE m-2 s-1), reference Brinkman et al., 2015, EMS-Dollard data report
hr_KNMI_deKooy_2021_30$Datum <- paste(hr_KNMI_deKooy_2021_30$YYYYMMDD, hr_KNMI_deKooy_2021_30$HH,sep=" ")%>%ymd_h
# plot(hr_KNMI_deKooy_2021_30$Datum[1:365], hr_KNMI_deKooy_2021_30$PAR[1:365])

hr_KNMI_deKooy <- rbind(hr_KNMI_deKooy_2011_20, hr_KNMI_deKooy_2021_30)

# load water height data from dtR by Karline
# Water heights are downloaded from a nearby station, OudeSchild (OUDSD, 4.850192, 53.03884).
# Check den helder water depth (program)
# load(paste(wd_path,"/processed_data/","WHeightJetty.rda",sep=""))

# WH_2015_17 <- subset(WHeightJetty,
#                      datetime>=as.POSIXct("2015-01-01") &
#                        datetime <= as.POSIXct("2017-12-31"))


# daily_HW <- WH_2015_17%>%group_by(datetime%>%date)%>%summarise(daily_height=mean(Height,na.rm = TRUE))

# names(daily_HW) <- c("datum","daily_height")

#### interpolate daily data to hourly data ####
# Create a data frame
daily_df <- merge(daily_df, daily_PIdata, by=c("Year","Day","Date"))

# Generate hourly timestamps
time_period <- paste(range(daily_df$Date),"00:00:00")%>%ymd_hms
hourly_dates <- seq.POSIXt(from = time_period[1], to = time_period[2], by = "hour")

# Interpolation function
interpolate_to_hourly <- function(daily_data, hourly_dates) {
  hourly_data <- data.frame(Date = hourly_dates)
  
  for (var in names(daily_data)[-1]) {
    # hourly_data[[var]] <- approx(x = as.numeric(daily_data$Date%>%as.POSIXct()), 
    #                              y = daily_data[[var]], 
    #                              xout = as.numeric(hourly_dates), 
    #                              method = "linear",rule=2)$y
    hourly_data[[var]] <- approx(x = as.numeric(daily_data$Date%>%as.POSIXct()), 
                                 y = daily_data[[var]], 
                                 xout = as.numeric(hourly_dates), 
                                 method = "linear",rule=2)$y
  }
  
  return(hourly_data)
}

# Interpolate data
hourly_data <- interpolate_to_hourly(daily_df, hourly_dates)
hourly_data$Date <- hourly_dates
# View the first few rows of the interpolated data
head(hourly_data)

# integration function to calculate PP based on light, Chla


# Define the primary production function as a function of depth


integrate_primary_production <- function(z) {
  #PAR :photosynthetically active radiation,μE m‐2 s‐1
  # Chla: mg Chla
  # a,b,c Eilers-Peeters PI curve, a, b, c parameters
  
  light_at_depth <- light_surface * exp(-k * z)*(1-0.07)# reflecting of solar radiatio nat the surface by 5-10%, ref Brinkman et al., 2015
  
  # Example primary production function (modify as needed)
  production <- (light_at_depth/(a*light_at_depth^2 + b*light_at_depth + c))*Chla # unit: PP (mg C m-2 h-1)  
  return(production)
}

# integrated_PP_daily = data.frame(datum=daily_PIdata$Date%>%ymd,PP=NA)
# depth_range = c(0, 3.3) # assuming 10 m constant depth, depth from Katja
# for (i in 1:nrow(integrated_PP_daily)) {
#   # Define the depth range for integration (e.g., from 0 to 100 meters)
#   
#   light_surface = hr_KNMI_deKooy$PAR[i]
#   a = daily_PIdata$a_fit[i]
#   b = daily_PIdata$b_fit[i]
#   c = daily_PIdata$c_fit[i]
#   k = daily_Kd$Kd_ave[i]
#   Chla = daily_Chla$Chla_ug_l[i] # unit conversion: 1 ug Chla/L = 1 mg/m3
#   water_depth = 4.6 #assuming fixed water depth = 4.6, ref. Jacobs et al., 2020
#   
#   integrated_PP_daily$PP[i] <-  integral(integrate_primary_production, 0, water_depth)
#   
# }

integrated_PP_hourly <- merge(hr_KNMI_deKooy[,c("Q","PAR","Datum","STN")], hourly_data[,c("Date","Chl","Kd_Secci","a_fit","b_fit","c_fit")],by.x="Datum", by.y="Date")
integrated_PP_hourly$PP <- NA
# integrated_PP_hourly = data.frame(datum=hourly_dates,PP=NA)
for (i in 1:nrow(integrated_PP_hourly)) {
  # Define the depth range for integration (e.g., from 0 to 100 meters)
  
  light_surface = integrated_PP_hourly$PAR[i]
  a = integrated_PP_hourly$a_fit[i]
  b = integrated_PP_hourly$b_fit[i]
  c = integrated_PP_hourly$c_fit[i]
  k = integrated_PP_hourly$Kd_Secci[i]
  Chla = integrated_PP_hourly$Chl[i] # unit conversion: 1 ug Chla/L = 1 mg/m3
  water_depth = 4.6 #assuming fixed water depth = 4.6, ref. Jacobs et al., 2020
  
  
  integrated_PP_hourly$PP[i] <-  integral(integrate_primary_production, 0, water_depth)
  
}

sumDay_PP <- integrated_PP_hourly%>%
  mutate(date=as.Date(Datum))%>%
  group_by(date)%>%
  summarize(daily_PP=sum(PP,na.rm=T))

plot(sumDay_PP$date,sumDay_PP$daily_PP,ylab="production mgC/m2/d",t="l")
abline(h=mean(sumDay_PP$daily_PP),lty=2,col="darkgreen")
legend("topleft",legend = paste("average PP= ", trunc(mean(sumDay_PP$daily_PP)), "mgC/m2/day", sep=""),lty=2,col = "darkgreen",bg="grey93")

# write.csv(sumDay_PP, file=paste(wd_path,"/PP_cal/","sumDay_PP.csv",sep=""))


In [ ]:
## 3.1 *Fig*. PP & Chla dynamics (2012-2024)


require(ggplot2)
require(lubridate)
require(dplyr)
require(mgcv)  # For GAM models
library(patchwork)

# Load and prepare data
PI_curve <- read.csv(paste(PI_output_dir, "output_JP_v2.csv", sep = "")) %>%
  mutate(datum = ymd_hms(datum),
         I_opt = P_max / alpha)

Jetty_df <- read.csv(
  file = paste(wd_path, "/processed_data/", "Jetty_HWseries.csv", sep = ""),
  fileEncoding = "UTF-8",
  na.strings = c("", "NA"),
  header = TRUE
) %>%
  mutate(timestamp = ymd_hms(timestamp))

# Merge datasets
PI_Jetty <- merge(PI_curve, Jetty_df, by.x = "HWcode", by.y = "SampleID") %>%
  mutate(Date = as.Date(datum),
         day_of_year = yday(datum),
         DIN = NH4 + NO3 + NO2)

sumDay_PP_total <- read.csv(file = paste(wd_path, "/PP_cal/", "sumDay_PP_JP.csv", sep = "")) %>%
  mutate(date = ymd(as.character(date)))
sumYear_PP_total <- aggregate(sumDay_PP_total$daily_PP, by=list(year(sumDay_PP_total$date)),FUN=sum)
names(sumYear_PP_total) <- c("Year","PP_sumYear")
sumYear_PP_total <- sumYear_PP_total[which(sumYear_PP_total$Year!=2024),]

HW_PP <- merge(PI_Jetty, sumDay_PP_total, by.x = "Date", by.y = "date")

sumDay_PP_total$date <- ymd(sumDay_PP_total$date)
sumDay_PP_total$doy <- yday(sumDay_PP_total$date)

sumDay_PP_total$daily_PP[sumDay_PP_total$daily_PP <= 0] <- 0.001

gam_model <- gam(daily_PP ~ s(doy, bs = "tp", k=10), 
                 family = Gamma(link = "log"), 
                 data = sumDay_PP_total)

gam_pred <- predict(gam_model, 
                    newdata = sumDay_PP_total,
                    type="link",se.fit = TRUE)

# Predict values using the fitted model
sumDay_PP_total <- sumDay_PP_total%>%
  mutate(
    Fitted = exp(gam_pred$fit),
    lower_ci = exp(gam_pred$fit - 1.96*gam_pred$se.fit),
    upper_ci = exp(gam_pred$fit + 1.96*gam_pred$se.fit)
  )

summary(gam_model) # 0.405   
AIC(gam_model)

# ==================== PANEL A: TIME SERIES ====================
library(ggplot2)
library(patchwork)

# ==================== PANEL A: PRIMARY PRODUCTION ====================
PP_ts <- ggplot(sumDay_PP_total, aes(x = date, y = daily_PP)) +
  geom_line(linewidth = 0.5, color = "#1E88E5") +
  geom_point(data = HW_PP, aes(x = Date, y = daily_PP), 
             size = 1.5, shape = 21, fill = "#FFC107", color = "black", stroke = 0.3) +
  labs(
    y = expression(Primary~Production~(mg~C~m^{-2}~day^{-1})),
    x = NULL
  ) +
  scale_x_date(
    date_breaks = "2 years",
    date_labels = "%Y",
    expand = c(0.02, 0.02)
  ) +
  scale_y_continuous(
    expand = expansion(mult = c(0, 0.1)),
    limits = c(0, NA)
  ) 


Nut_Light_limitation_df <- read.csv(paste(PP_cal_dir,"Nut_Light_limitation_Jetty40.csv",sep=""))
Nut_Light_limitation_df$Lim_Nut <- factor(Nut_Light_limitation_df$Lim_Nut, levels=c("DIN","DP","DSi"))
Nut_Light_limitation_df$date<-Nut_Light_limitation_df$date%>%as.Date()


# ==================== PANEL A: TIME SERIES ====================
library(ggplot2)
library(patchwork)

# Merge the two datasets by date
df_combined <- merge(
  PI_Jetty[, c("Date", "Chl")],
  Nut_Light_limitation_df[, c("date", "mu")],
  by.x = "Date", by.y = "date",
  all = TRUE
)

# Reverse Chl range and get growth rate range
chl_range <- range(df_combined$Chl, na.rm = TRUE)*1.1
mu_range  <- range(df_combined$mu, na.rm = TRUE)*1.1

# NOTE: since we reverse Chl visually, its lower value is plotted higher.
# So: match min(Chl) to top and max(Chl) to bottom.

# Rescaling factor for mu → reversed Chl scale
scale_factor <- diff(rev(chl_range)) / diff(mu_range)

# Rescale growth rate to reversed Chl scale
df_combined$mu_rescaled <- (df_combined$mu - mu_range[1]) * scale_factor + chl_range[2]

# Plot
Chla_growth_ts <- ggplot(df_combined, aes(x = Date)) +
  # Chl-a (left axis, reversed)
  geom_line(aes(y = Chl), linewidth = 0.5, color = "#006400") +
  geom_point(aes(y = Chl), size = 1.5, shape = 21, fill = "#006400", color = "black", stroke = 0.3) +
  # Growth rate (on rescaled scale)
  geom_line(aes(y = mu_rescaled), linewidth = 0.5, color = "#E63946") +
  geom_point(aes(y = mu_rescaled), size = 1.5, shape = 21, fill = "#E63946", color = "black", stroke = 0.3) +
  labs(
    y = expression("Biomass (mg Chl-a m"^{-3}*")"),
    x = NULL
  ) +
  scale_x_date(
    date_breaks = "2 years",
    date_labels = "%Y",
    expand = c(0.02, 0.02)
  ) +
  scale_y_reverse(  # primary (left) axis: Chl-a reversed
    expand = expansion(mult = c(0, 0.1)),
    sec.axis = sec_axis(
      # rescale from reversed Chl scale back to growth rate
      ~ (.- chl_range[2]) / scale_factor + mu_range[1],
      name = expression("Growth rate ("*d^{-1}*")")
    )
  ) 

Chla_growth_ts <- Chla_growth_ts +
  theme(
    axis.title.y = element_text(color = "#006400", size = 10),
    axis.title.y.right = element_text(color = "#E63946", size = 10)
  )






# ==================== COMBINE WITH PATCHWORK AND TAGS ====================
combined_plot <- PP_ts / Chla_growth_ts +
  plot_annotation(tag_levels = "A", tag_prefix = "(", tag_suffix = ")") &
  theme(
    plot.tag = element_text(face = "bold", hjust = 0, vjust = 1)
  )

combined_plot

# ==================== SAVE OUTPUT ====================
#ggsave(
#  filename = paste0(graph_dir, "PrimaryProduction_biomass_growth_Fig.tiff"),
#  plot = combined_plot,
#  device = "tiff",
#  width = 180,  # mm (standard width for 1-column figures)
#  height = 150, # mm
#  units = "mm",
#  dpi = 600,
#  compression = "lzw",
#  bg="white"
#)


# Compute ratio and add to dataframe
PI_Jetty$Date <- as.Date(PI_Jetty$Date)
PI_Jetty$PP_Chl_ratio <- HW_PP$daily_PP / PI_Jetty$Chl

PP_Chl_ts <- ggplot(PI_Jetty, aes(x = Date, y = PP_Chl_ratio)) +
  geom_line(linewidth = 0.5, color = "#E63946") +
  geom_point(aes(x = Date, y = PP_Chl_ratio), 
             size = 1.5, shape = 21, fill = "#FDB813", color = "black", stroke = 0.3) +
  labs(
    y = expression("PP / Biomass ("*mg~C~m^{-3}~h^{-1}~"/"~mu*g~Chl*a~L^{-1}*")"),
    x = NULL
  ) +
  scale_x_date(
    date_breaks = "2 years",
    date_labels = "%Y",
    expand = c(0.02, 0.02)
  ) +
  scale_y_continuous(
    expand = expansion(mult = c(0, 0.1))
  )

PP_Chl_ts
# ggsave(
#   filename = paste0(graph_dir, "PrimaryProduction_BiomassRatio_Fig.png"),
#   plot = PP_Chl_ts,
#   device = "png",
#   width = 180,  # mm (standard width for 1-column figures)
#   height = 75, # mm
#   units = "mm",
#   dpi = 600,
#   bg = "white"
# )

# Or, if focusing on 2018 onward:
PI_Jetty_post2018 <- PI_Jetty %>% filter(year_frac >= 2018)

# Add a fractional year column
PI_Jetty$year <- year(PI_Jetty$datum)
# + (yday(PI_Jetty$datum) - 1) / 365
model_full <- lm(PP_Chl_ratio ~ year, data = PI_Jetty)
summary(model_full)

PI_Jetty$PP_Chl_ratio[which(PI_Jetty$year==2018)]%>%mean

PI_Jetty$post2018 <- ifelse(PI_Jetty$datum%>%year > 2018, "Y", "N")

lm(Chl~post2018, data = PI_Jetty)%>%summary

Nut_Light_limitation_df$post2018  <- ifelse(Nut_Light_limitation_df$year > 2018, "Y", "N")
lm(mu~post2018, data = Nut_Light_limitation_df)%>%summary
aov(mu~post2018, data = Nut_Light_limitation_df)%>%summary


In [ ]:
## 3.2 *Fig* Seanality of PP & Biomass

require(dplyr)
require(ggplot2)
library(patchwork)
library(mgcv)
library(ggplot2)
library(lubridate)
library(gridExtra)
library(plotly)
library(pracma)

Nut_Light_limitation_df <- read.csv(paste(PP_cal_dir,"Nut_Light_limitation_Jetty40.csv",sep=""))
Nut_Light_limitation_df$Lim_Nut <- factor(Nut_Light_limitation_df$Lim_Nut, levels=c("DIN","DP","DSi"))

# load Eilers-Peeters a, b, c parameters
PI_curve <- read.csv(paste(PI_output_dir,"output_v2.csv",sep=""))
PI_curve <- PI_curve[,c("datum", "HWcode", "a_fit", "b_fit", "c_fit")]
PI_curve$P_max = 1/(2*sqrt(PI_curve$a_fit*PI_curve$c_fit)+PI_curve$b_fit) # maximum productivity in mg C/mg chla/h
PI_curve$I_opt = sqrt(PI_curve$c_fit/PI_curve$a_fit) # optimum light intensity in uE/m2/s
PI_curve$initial_slop = 1/PI_curve$c_fit

PI_curve$datum <- PI_curve$datum%>%ymd_hms


# Plotting PP against day of years
sumDay_PP_total <- read.csv(file = paste(wd_path,"/PP_cal/","sumDay_PP.csv",sep=""))
sumDay_PP_total$date <- ymd(sumDay_PP_total$date)
sumDay_PP_total$doy <- yday(sumDay_PP_total$date)

sumDay_PP_total$daily_PP[sumDay_PP_total$daily_PP <= 0] <- 0.001

Jetty_df = read.csv(file=paste(wd_path,"/processed_data/","Jetty_HWseries.csv",sep=""), fileEncoding = "UTF-8", na.strings=c("", "NA"), header=TRUE)
Jetty_df$timestamp <- Jetty_df$timestamp%>%ymd_hms()
Jetty_df$doy <- yday(Jetty_df$timestamp)
Jetty_df$Day <- yday(Jetty_df$timestamp)
Jetty_df$Year <- year(Jetty_df$timestamp)
Jetty_df$DIN <- Jetty_df$NH4 + Jetty_df$NO2 + Jetty_df$NO3

Chla_1974_2012 = read.csv(file=paste(wd_path,"/rawdata/","Chla_CovertHPLC_1974_2012.csv",sep=""), fileEncoding = "UTF-8", na.strings=c("", "NA"), header=TRUE)
Chla_1974_2012$date_descr <- dmy(Chla_1974_2012$date_descr)

Jetty_df$Chl[which(Jetty_df$Year>=1990&Jetty_df$Year<=2011)] = Chla_1974_2012$HPLC_convmeas[which(year(Chla_1974_2012$date_descr)>=1990&year(Chla_1974_2012$date_descr)<=2011)] # replace Spectrophotometry-Chla with Converted HPLC-Chla (based on emperical relationship, HPLC-C ~ 0.83 * Spect-Chla)

gam_model <- gam(daily_PP ~ s(doy, bs = "tp"), 
                 family = Gamma(link = "log"), 
                 data = sumDay_PP_total)

gam_pred <- predict(gam_model, 
                    newdata = sumDay_PP_total,
                    type="link",se.fit = TRUE)

# Predict values using the fitted model
sumDay_PP_total <- sumDay_PP_total%>%
  mutate(
    Fitted = exp(gam_pred$fit),
    lower_ci = exp(gam_pred$fit - 1.96*gam_pred$se.fit),
    upper_ci = exp(gam_pred$fit + 1.96*gam_pred$se.fit)
  )



Jetty_df_2012_2023 <- Jetty_df[which(Jetty_df$Year>=2012&Jetty_df$Year<=2023),]
Chl_gam_model <- gam(Chl ~ s(doy, bs = "tp"), 
                     family = Gamma(link = "log"), 
                     data = Jetty_df_2012_2023)

Chl_gam_pred <- predict(Chl_gam_model, 
                        newdata = Jetty_df_2012_2023,
                        type="link",se.fit = TRUE)

# Predict values using the fitted model
sumDay_PP_total <- sumDay_PP_total%>%
  mutate(
    Fitted = exp(gam_pred$fit),
    lower_ci = exp(gam_pred$fit - 1.96*gam_pred$se.fit),
    upper_ci = exp(gam_pred$fit + 1.96*gam_pred$se.fit)
  )

Jetty_df_2012_2023 <- Jetty_df_2012_2023%>%
  mutate(
    Fitted = exp(Chl_gam_pred$fit),
    lower_ci = exp(Chl_gam_pred$fit - 1.96*Chl_gam_pred$se.fit),
    upper_ci = exp(Chl_gam_pred$fit + 1.96*Chl_gam_pred$se.fit)
  )

summary(gam_model) # 0.405   
AIC(gam_model)

summary(Chl_gam_model)
AIC(Chl_gam_model)

sumDay_PP_doy <- sumDay_PP_total%>%group_by(doy)%>%summarize(Fitted_PP = unique(Fitted))

sumDay_PP_doy <- sumDay_PP_doy %>% arrange(doy) %>% mutate(DailyChange=c(NA, diff(Fitted_PP)), DailyChangePercent = c(NA, diff(Fitted_PP)/Fitted_PP[-length(Fitted_PP)]*100))%>%na.omit

sumDay_PP_doy$DailyChangePercent %>% which.min
sumDay_PP_doy$DailyChangePercent %>% which.max

p0 <- ggplot(sumDay_PP_total, aes(x = doy)) +
  # Raw data points
  geom_point(aes(y = daily_PP), 
             shape = 21, fill = "steelblue", color = "white", 
             size = 2, alpha = 0.7, stroke = 0.5) +
  # Fitted line
  geom_line(aes(y = Fitted), 
            color = "#E69F00", size = 1.2, lineend = "round") +
  # Vertical reference lines
  geom_vline(xintercept = c(sumDay_PP_doy$doy[which.max(sumDay_PP_doy$DailyChangePercent)],
                            sumDay_PP_doy$doy[which.min(sumDay_PP_doy$DailyChangePercent)]), 
             linetype = "dashed", color = "gray30", size = 0.8) +
  geom_ribbon(aes(ymin = lower_ci, ymax = upper_ci), 
              fill = "grey", alpha = 0.8) +
  labs(title = "Daily primary production over day of year (2012-2023)",
       y = "daily primary production (mgC/m²/d)") +
  # Axis scales
  scale_x_continuous(
    limits = c(1, 365),
    breaks = seq(15, 345, by = 30),  # Center month labels
    labels = month.abb[1:12],        # Standard month abbreviations
    expand = c(0.01, 0.01)          # Tighten axis padding
  )  + theme_minimal() +
  theme(
    text = element_text(family = "sans"),
    axis.title.x = element_blank(),
    axis.text.x = element_text(size = 10, color = "black"),
    axis.title.y = element_text(size = 12, face = "bold"),
    axis.text.y = element_text(size = 10),
    panel.grid.major = element_line(color = "gray90", linewidth = 0.25),
    panel.grid.minor = element_blank(),
    plot.title = element_blank()  # Removed as caption typically goes in manuscript
  )

# ggsave(
#   filename = file.path(graph_dir, "PP_GAM_ts.png"),
#   plot = p0,
#   device = "png",
#   width = 7,       # Standard single-column width (inches)
#   height = 5,        # Adjusted height for better proportions
#   units = "in",
#   dpi = 600,         # Publication quality
#   bg = "white"
# )

# Color-blind safe palette
pp_color <- "#FFC107"

chla_color <- "#006400"   # Blue for Chla
pp_ribbon <- "#F2C6B4"    # Soft orange tint
chla_ribbon <- "#BFD3E6"  # Soft blue tint

# Normalize Pilot_chla for plotting on same axis
range_PP <- range(sumDay_PP_total$Fitted, na.rm = TRUE)
range_chla <- range(Jetty_df_2012_2023$Fitted, na.rm = TRUE)

# Scale function for plotting
scale_chla_to_PP <- function(x) {
  (x - min(range_chla)) / diff(range_chla) * diff(range_PP) + min(range_PP)
}

# Inverse scale function for secondary axis
scale_PP_to_chla <- function(x) {
  (x - min(range_PP)) / diff(range_PP) * diff(range_chla) + min(range_chla)
}

# Add Pilot_chla to PP_plot
PP_plot <- ggplot(sumDay_PP_total, aes(x = doy)) +
  xlim(1, 365) +
  
  # Error ribbon for PP
  geom_ribbon(data = sumDay_PP_total, 
              aes(ymin = lower_ci, ymax = upper_ci), 
              fill = pp_ribbon, alpha = 0.6) +
  
  # PP line
  geom_line(data = sumDay_PP_total, 
            aes(y = Fitted), 
            color = pp_color, size = 2) +
  
  # Error ribbon for scaled chla
  geom_ribbon(data = Jetty_df_2012_2023, 
              aes(ymin = scale_chla_to_PP(lower_ci), ymax = scale_chla_to_PP(upper_ci)), 
              fill = chla_ribbon, alpha = 0.6) +
  
  # Chla line
  geom_line(data = Jetty_df_2012_2023, 
            aes(x = doy, y = scale_chla_to_PP(Fitted)), 
            color = chla_color, size = 1.2) +
  
  # Y-axes
  scale_y_continuous(
    name = expression("Primary production (" * mg~C~m^{-2}~d^{-1} * ")"),
    sec.axis = sec_axis(~scale_PP_to_chla(.), 
                        # name = expression("Biomass, Chl-a (" * mg~m^{-3} * ")")
                        name = expression("Biomass (mg Chl-a m"^{-3}*")")
                          )
  ) +
  theme_minimal() +
  theme(
    # axis.title.x = element_blank(),
    # axis.text.x = element_blank(),
    axis.title.y.left = element_text(size = 14, face = "bold", color = pp_color),
    axis.title.y.right = element_text(size = 14, face = "bold", color = chla_color),
    axis.text.y.left = element_text(size = 11, color = pp_color),
    axis.text.y.right = element_text(size = 11, color = chla_color),
    plot.title = element_text(size = 16, face = "bold", hjust = 0.5),
    panel.grid.major = element_line(color = "gray90"),
    panel.grid.minor = element_blank()
  )
PP_plot

# Save the plot as a TIFF file
# tiff(filename = paste0(graph_dir, "PP_Chla_Timeseries.tiff"), 
#     width = 3000, height = 2000, res = 300)  # 300 dpi, suitable for publication
# print(PP_plot)
# dev.off()


In [ ]:
# 4. Light vs nutrient limitation
## 4.1 Cloern approach using 40

require(dplyr)
require(ggplot2)
library(lubridate)
library(ggplot2)
library(zoo)

# load Eilers-Peeters a, b, c parameters
PI_curve <- read.csv(paste(PI_output_dir,"output_v2.csv",sep=""))
PI_curve <- PI_curve[,c("datum", "HWcode", "a_fit", "b_fit", "c_fit")]
PI_curve$P_max = 1/(2*sqrt(PI_curve$a_fit*PI_curve$c_fit)+PI_curve$b_fit) # maximum productivity in mg C/mg chla/h
PI_curve$I_opt = sqrt(PI_curve$c_fit/PI_curve$a_fit) # optimum light intensity in uE/m2/s
PI_curve$initial_slop = 1/PI_curve$c_fit

PI_curve$datum <- PI_curve$datum%>%ymd_hms
names(PI_curve)[3:5] = c("PI_a", "PI_b", "PI_c")

#plot(PI_curve$datum, PI_curve$P_max, ylim=c(0,40), pch=19, ylab="Max productivity (mg C/mg chla/h)")
#plot(PI_curve$datum, PI_curve$I_opt, ylim=quantile(PI_curve$I_opt%>%na.omit, c(0,0.99)), pch=19, ylab="optimum light in uE/m2/s")
#plot(PI_curve$datum, PI_curve$initial_slop, ylim=quantile(PI_curve$initial_slop%>%na.omit, c(0,0.99)), pch=19, ylab="initial slope in mgC/mgChla/h (uE/m2/s)-1")

Jetty_df = read.csv(file=paste(wd_path,"/processed_data/","Jetty_HWseries.csv",sep=""), fileEncoding = "UTF-8", na.strings=c("", "NA"), header=TRUE)
Jetty_df$timestamp <- Jetty_df$timestamp%>%ymd_hms()


#plot(Jetty_df$timestamp,Jetty_df$PO4, ylab=c("PO4 (ug/L)"), t="b")
#plot(Jetty_df$timestamp,Jetty_df$NO2+Jetty_df$NO3+Jetty_df$NH4, ylab=c("DIN (ug/L)"), t="b")
#plot(Jetty_df$timestamp,Jetty_df$Si, ylab=c("DIN (ug/L)"), t="b")

PI_Jetty <- merge(PI_curve, Jetty_df, by.x="HWcode", by.y="SampleID")
PI_Jetty$DIN <- PI_Jetty$NH4 + PI_Jetty$NO3 + PI_Jetty$NO2

# Load necessary libraries
# Add a new column for the day of the year
PI_Jetty$day_of_year <- yday(PI_Jetty$datum)

PI_Jetty <- PI_Jetty %>%
  mutate(date = as.Date(datum),
         year = year(datum),
         month = month(datum)
  )

#######
# KNMI measurements
#######

hr_KNMI_filepath_2011_2020 <- paste(wd_path,"/rawdata/","uurgeg_235_2011-2020.txt",sep="")
hr_KNMI_filepath_2021_2030 <- paste(wd_path,"/rawdata/","uurgeg_235_2021-2030.txt",sep="")

hr_KNMI_deKooy_2011_20 <- read.table(hr_KNMI_filepath_2011_2020,header=TRUE, sep=",")
hr_KNMI_deKooy_2021_30 <- read.table(hr_KNMI_filepath_2021_2030, header=TRUE, sep=",")

hr_KNMI_deKooy <- rbind(hr_KNMI_deKooy_2011_20, hr_KNMI_deKooy_2021_30)

# hr_KNMI_deKooy$PAR = hr_KNMI_deKooy$Q*1e4/3600*2.515 # Conversion from KNMI 1J/(W m-2) to PAR (uE m-2 s-1), reference Brinkman et al., 2015, EMS-Dollard data report
hr_KNMI_deKooy$Datum <- paste(hr_KNMI_deKooy$YYYYMMDD, hr_KNMI_deKooy$HH,sep=" ")%>%ymd_h
hr_KNMI_deKooy <- hr_KNMI_deKooy[which(year(hr_KNMI_deKooy$Datum)%in%2012:2023),]

PAR_frac = 0.45  # PAR is 45% of global radiation
Reflec_frac = 0.07 # reflecting of solar radiation at the surface by 5-10%, ref Brinkman et al., 2015
hr_KNMI_deKooy$PAR = hr_KNMI_deKooy$Q*PAR_frac*(1-Reflec_frac) # in (J/cm2/h) over an hour at water surface

# convert to photon flux (uE m-2 s-1); reference Brinkman et al., 2015,    EMS-Dollard data report
sec_per_hr = 3600 # seconds in an hour
cm2_to_m2 = 1e4 
conversion_factor = 4.66 # (μE/m2/s)/(W/m2); reference Jacobs et al., 2021, https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0246012 (in Brinkman et al., 2015 data report this conversion factor is 2.515)

day_ave_PAR <- aggregate(hr_KNMI_deKooy$PAR, by = list(as.Date(hr_KNMI_deKooy$Datum)),FUN=mean) # in (J/cm2/h)
names(day_ave_PAR) <- c("date","PAR")
day_ave_PAR$I_surf <- day_ave_PAR$PAR/3600*3600*24*cm2_to_m2*conversion_factor*1e-6 # in (mol E/m2/day)

plot(day_ave_PAR$date, day_ave_PAR$I_surf, ylab="(mol E/m2/day)")

# Add a month column
day_ave_PAR <- day_ave_PAR[,c("date","I_surf")]
names(day_ave_PAR) <- c("date","irradiance")
day_ave_PAR <- day_ave_PAR %>%
  mutate(year = year(date),
         month = month(date)
  )


Nut_Light_limitation_df = merge(PI_Jetty,day_ave_PAR[,c("date","irradiance")], by.x = c("date"), by.y=c("date"))
Nut_Light_limitation_df = Nut_Light_limitation_df[,-which(names(Nut_Light_limitation_df)=="timestamp")]

# Define the integrate_primary_production function
integrate_primary_production <- function(z, PAR_daily, k, a, b, c) {
  light_hourly <- PAR_daily * 1e6 / (24 * 3600) # Convert from mol E/m2/day to umol E/m2/s
  light_at_depth <- light_hourly * exp(-k * z)
  production <- (light_at_depth / (a * light_at_depth^2 + b * light_at_depth + c)) * 24 # PP in mg C/m2/day
  return(production)
}

# Calcualate the average light over depth:
I_ave_depth <- function(PAR_daily, k, z) {
  I_mean_day_over_depth <- (PAR_daily / (k * z)) * (1 - exp(-(k * z)))
}

# Define the growth rate function

growth_rate <- function(PAR_daily, nutrient, Temp, k, a, b, c, Kn, z) {
  I_mean_day_over_depth <- (PAR_daily / (k * z)) * (1 - exp(-(k * z)))
  Chla_C <- 0.003 + 0.0154 * exp(0.050 * Temp) * exp(-0.059 * I_mean_day_over_depth) * (nutrient / (nutrient + Kn))
  
  # Depth-integrated primary production
  PP_over_depth <- integrate(integrate_primary_production, lower = 0, upper = z,
                             PAR_daily = PAR_daily, k = k, a = a, b = b, c = c)$value / z
  
  mu <- 0.85 * PP_over_depth * Chla_C - 0.015
  return(mu)
}

# Five-point formula for partial derivative with respect to light
partial_derivative_light <- function(light, nutrient, h, Temp, k, a, b, c, Kn, z) {
  (growth_rate(light * (1 - 2 * h), nutrient, Temp, k, a, b, c, Kn, z) -
     8 * growth_rate(light * (1 - h), nutrient, Temp, k, a, b, c, Kn, z) +
     8 * growth_rate(light * (1 + h), nutrient, Temp, k, a, b, c, Kn, z) -
     growth_rate(light * (1 + 2 * h), nutrient, Temp, k, a, b, c, Kn, z)) / (12 * h * light)
}

# Five-point formula for partial derivative with respect to nutrient
partial_derivative_nutrient <- function(light, nutrient, h, Temp, k, a, b, c, Kn, z) {
  (growth_rate(light, nutrient * (1 - 2 * h), Temp, k, a, b, c, Kn, z) -
     8 * growth_rate(light, nutrient * (1 - h), Temp, k, a, b, c, Kn, z) +
     8 * growth_rate(light, nutrient * (1 + h), Temp, k, a, b, c, Kn, z) -
     growth_rate(light, nutrient * (1 + 2 * h), Temp, k, a, b, c, Kn, z)) / (12 * h * nutrient)
}


Nut_Light_limitation_df <- Nut_Light_limitation_df %>%
  arrange(date) %>%
  mutate(
    across(
      .cols = -(1:3),  # all columns except the first two
      .fns  = ~ na.approx(., x = date, na.rm = FALSE)
    )
  )

# Nut_Light_limitation_df = na.omit(Nut_Light_limitation_df[,PP_input_vars])
names(Nut_Light_limitation_df)[which(names(Nut_Light_limitation_df)=="T")] <- "T_insitu" 
# Main loop for calculations

for (i in 1:nrow(Nut_Light_limitation_df)) {
  Date <- Nut_Light_limitation_df$date[i]
  Nuts <- c(DIN = Nut_Light_limitation_df$DIN[i], DSi = Nut_Light_limitation_df$Si[i], DP = Nut_Light_limitation_df$PO4[i])
  # Knuts <- c(K_N = 1.5, K_Si = 1.5, K_P = 0.15) # Cloern 1999 for K_N, and philippart et al., (2007) for the ratio between K_N/K_Si=1, K_N/K_P=10
  Knuts <- c(K_N = 2, K_Si = 2, K_P = 0.2) # Philippart et al., (2007)
  # Knuts <- c(K_N = 1.5, K_Si = 5, K_P = 0.5) # Loebl et al., (2009); attention they altered the procedure of Cloern and took the monthly mean irradiance, instead of the average value from the 3 days around the middle of each month to reduce short-term variability.
  
  Nut.Lim.ind <- which.min(c(Nuts["DIN"] / (Nuts["DIN"] + Knuts["K_N"]),
                             Nuts["DSi"] / (Nuts["DSi"] + Knuts["K_Si"]),
                             Nuts["DP"] / (Nuts["DP"] + Knuts["K_P"])))
  Nut_Light_limitation_df$Lim_Nut[i] <- names(Nuts)[Nut.Lim.ind]
  Nut <- Nuts[Nut.Lim.ind];names(Nut)<-NULL
  Kn <- Knuts[Nut.Lim.ind];names(Kn)<-NULL
  
  parameters <- c(Temp = Nut_Light_limitation_df$T_insitu[i],
                  k = Nut_Light_limitation_df$Kd_Secci[i],
                  a = Nut_Light_limitation_df$PI_a[i],
                  b = Nut_Light_limitation_df$PI_b[i],
                  c = Nut_Light_limitation_df$PI_c[i],
                  PAR_daily = Nut_Light_limitation_df$irradiance[i],
                  Nut = Nut,
                  Kn = Kn,
                  K_I = 2.4, # mol E/m2/d, based on Cloern 1999
                  z = 4.6)
  
  # Nut_Light_limitation_df$mu[i] <- with(as.list(parameters), {
  #   return(growth_rate(PAR_daily = PAR_daily, nutrient = Nut, Temp = Temp, k = k, a = a, b = b, c = c, Kn = Kn, z = z))
  # })
  Nut_Light_limitation_df$mu[i] =
    growth_rate(PAR_daily = parameters["PAR_daily"], nutrient = parameters["Nut"], Temp = parameters["Temp"], k = parameters["k"], a = parameters["a"], b = parameters["b"], c = parameters["c"], Kn = parameters["Kn"], z = parameters["z"])
  
  Nut_Light_limitation_df$I_depth[i] = I_ave_depth(PAR_daily = parameters["PAR_daily"], k = parameters["k"], z = parameters["z"])
  Nut_Light_limitation_df$nutrient_limitation[i] = min(c(Nuts["DIN"] / (Nuts["DIN"] + Knuts["K_N"]),
                                                               Nuts["DSi"] / (Nuts["DSi"] + Knuts["K_Si"]),
                                                               Nuts["DP"] / (Nuts["DP"] + Knuts["K_P"])))
  
  
  light_nonDIM <- parameters["PAR_daily"] / parameters["K_I"]
  Nut_nonDIM <- parameters["Nut"] / parameters["Kn"]
  
  Nut_Light_limitation_df$partial_dev_light[i] <- partial_derivative_light(
    light = light_nonDIM, nutrient = Nut_nonDIM, h = 0.0025,
    Temp = parameters["Temp"], k = parameters["k"], a = parameters["a"],
    b = parameters["b"], c = parameters["c"], Kn = parameters["Kn"], z = parameters["z"]
  )
  
  Nut_Light_limitation_df$partial_dev_nutrient[i] <- partial_derivative_nutrient(
    light = light_nonDIM, nutrient = Nut_nonDIM, h = 0.0025,
    Temp = parameters["Temp"], k = parameters["k"], a = parameters["a"],
    b = parameters["b"], c = parameters["c"], Kn = parameters["Kn"], z = parameters["z"]
  )
  
  Nut_Light_limitation_df$ln_R_light_to_nutrient[i] <- log(Nut_Light_limitation_df$partial_dev_light[i] / Nut_Light_limitation_df$partial_dev_nutrient[i])
}

Nut_Light_limitation_df$Lim_Nut <- factor(Nut_Light_limitation_df$Lim_Nut, levels=c("DIN","DSi","DP"))


ggplot(Nut_Light_limitation_df, aes(x = date, y = ln_R_light_to_nutrient)) +
  geom_line(aes(group=1),color="black", size = 0.8) +
  geom_point(aes(size = 3, shape = Lim_Nut, color = Lim_Nut)) +  # Adjust point size as needed
  geom_hline(yintercept = c(log(10), log(1),  log(0.1)), linetype = "dashed", color = "gray50", size = 0.5) +
  # # Add shaded regions for growing seasons
  # geom_rect(data = growing_seasons, aes(xmin = start, xmax = end, ymin = -Inf, ymax = Inf, fill = "gray",alpha=0.8),
  #           alpha = 0.3, inherit.aes = FALSE) +
  # # Add text annotations
  # annotate("text", x = mean(Date), y = mean(c(log(10),log(100))), label = "R > 10 -> strong light limitation", 
  #          hjust = 1.1, vjust = -0.5, color = "black", size = 8) +
  # annotate("text", x = mean(Date), y = mean(c(log(1),log(10))), label = "R (1-10) -> light limitation", 
  #          hjust = 1.1, vjust = -0.5, color = "black", size = 8) +
  # annotate("text", x = mean(Date), y = mean(c(log(0.1),log(1))), label = "R (0.1-1) -> nutrient limitation", 
  #          hjust = 1.1, vjust = -0.5, color = "black", size = 8) +
  # annotate("text", x = mean(Date), y = mean(c(log(0.01),log(0.1))), label = "R < 0.1 -> strong nutrient limitation", 
  #          hjust = 1.1, vjust = -0.5, color = "black", size = 8) +
  labs(title = "The ratio of growth-rate senstivity to light and nutrients",
       x = "",
       y = "ln R (light senstivity/nut senstivity)",
       shape = "Limiting nutrients",
       color = "Limiting nutrients", size=1.5) 
# + theme_minimal()  # Use a clean theme


Nut_Light_limitation_df$day_of_year = yday(Nut_Light_limitation_df$date)
Nut_Light_limitation_df$year <- as.factor(year(Nut_Light_limitation_df$date))
Nut_Light_limitation_df$Lim_Nut <- factor(Nut_Light_limitation_df$Lim_Nut, levels=c("DIN","DP","DSi"))
Nut_Light_limitation_df <- Nut_Light_limitation_df%>%
  mutate(limiting_factor= case_when(
    ln_R_light_to_nutrient > 0 ~ "light",
    TRUE ~ Lim_Nut
  ))
Nut_Light_limitation_df <- Nut_Light_limitation_df%>%
  mutate(
    line_thickness = case_when(
      limiting_factor == "light" ~ ln_R_light_to_nutrient,
      limiting_factor %in% c("DIN", "DP", "DSi") ~ abs(ln_R_light_to_nutrient)
    ))

shape_mapping <- c("light"=16, #circle
                   "DIN" = 15, # square
                   "DP" = 17, # triangle
                   "DSi" = 18) # diamond
#  

p_cloern <-ggplot(Nut_Light_limitation_df, aes(x = day_of_year, y = year)) +
  geom_point(aes(shape = limiting_factor, 
                 color = limiting_factor,
                 size = line_thickness),
             alpha = 0.5)+
  scale_shape_manual(values = shape_mapping) + 
  scale_color_manual(values = c("light" = "goldenrod",
                                "DIN" = "darkblue",
                                "DP" = "purple",
                                "DSi" = "green4")) +
  scale_size_continuous(
    name = "Limitation Strength\n(0 = Co-limitation, \n4 = Strong limitation)",
    range = c(0.5, 4),  # Adjusted minimum size for better visibility
    breaks = c(0.5, 2, 4),
    labels = c("Co-limitation", "Moderate", "Strong limitation")
  ) +
  scale_x_continuous(limits = c(1, 365),
                     breaks = seq(0, 360, by = 30),
                     labels = c("Jan", "Feb", "Mar", "Apr", "May", "Jun",
                                "Jul", "Aug", "Sep", "Oct", "Nov", "Dec", "")) +
  labs(
    x = "Months",
    y = "Year",
    title = "Light vs Nutrient limitations (Cloern approach)",
    subtitle = "Marsdiep (2012-2022)"
  ) +
  theme_minimal() +
  theme(
    axis.text = element_text(size = 12, face = "bold"),
    axis.title = element_text(size = 12, face = "bold"),
    plot.margin = margin(t = 0, unit = "pt"),
    # panel.grid.major = element_blank(),
    # panel.grid.minor = element_blank(),
    legend.position = "right",
    legend.box = "vertical",
    legend.title = element_text(size = 12, face = "bold"),
    legend.text = element_text(size = 12)
  ) +
  guides(
    shape = guide_legend(override.aes = list(size = 3)),
    color = guide_legend(override.aes = list(size = 3)),
    size = guide_legend()  # Leave as is, or customize if needed
  )
p_cloern

# write.csv(Nut_Light_limitation_df, file = paste(PP_cal_dir,"Nut_Light_limitation_Jetty40.csv",sep=""),row.names = F)

In [ ]:
## 4.1 *Fig*: Interannual variation of limitations
### Jetty 40

Nut_Light_limitation_df <- read.csv(paste(PP_cal_dir,"Nut_Light_limitation_Jetty40.csv",sep=""))
Nut_Light_limitation_df$Lim_Nut <- factor(Nut_Light_limitation_df$Lim_Nut, levels=c("DIN","DP","DSi"))

shape_mapping <- c("light"=16, #circle
                   "DIN" = 15, # square
                   "DP" = 17, # triangle
                   "DSi" = 18) # diamond
#  

library(ggplot2)
library(scales)

# Create the plot with manuscript-ready formatting
p_cloern <- ggplot(Nut_Light_limitation_df, aes(x = day_of_year, y = year)) +
  geom_point(
    aes(
      shape = limiting_factor,
      color = limiting_factor,
      size = line_thickness
    ),
    alpha = 0.7,  # Slightly increased alpha for better visibility in print
    stroke = 0.8  # Adds outline to points for better definition
  ) +
  scale_shape_manual(
    name = "Limiting Factor",
    values = shape_mapping,
    guide = guide_legend(override.aes = list(size = 3))
  ) + 
  scale_color_manual(
    name = "Limiting Factor",
    values = c(
      "light" = "#FFD700",  # Brighter gold for print visibility
      "DIN" = "#1F77B4",    # Modified blue
      "DP" = "#9467BD",     # Modified purple
      "DSi" = "#2CA02C"     # Modified green
    ),
    guide = guide_legend(override.aes = list(size = 3))
  ) +
  scale_size_continuous(
    name = "Limitation Strength",
    range = c(0, 5),  # Slightly increased size range
    breaks = c(1, 2.5, 4),
    labels = c("Co-limitation", "Moderate", "Strong")
  ) +
  scale_x_continuous(
    limits = c(1, 365),
    breaks = seq(15, 345, by = 30),  # Center month labels
    labels = month.abb[1:12],        # Use built-in month abbreviations
    expand = c(0.01, 0.01)          # Tighten axis padding
  ) +
  scale_y_continuous(
    breaks = unique(Nut_Light_limitation_df$year),  # Show all years
    expand = c(0.05, 0.05)
  ) +
  labs(
    x = NULL,  # Remove axis title (months are self-explanatory)
    y = "Year"
    # ,
    # title = "Seasonal Nutrient and Light Limitation Patterns",
    # subtitle = "Marsdiep (2012-2023)"
  ) +
  theme_minimal(base_size = 10) +
  theme(
    text = element_text(family = "sans"),  # Use journal-preferred font
    axis.text = element_text(size = 9, color = "black"),
    axis.title.y = element_text(size = 10, face = "bold", margin = margin(r = 10)),
    axis.title.x = element_blank(),
    plot.title = element_text(size = 11, face = "bold", hjust = 0.5),
    plot.subtitle = element_text(size = 10, hjust = 0.5, margin = margin(b = 10)),
    panel.grid.major = element_line(color = "gray90", linewidth = 0.2),
    panel.grid.minor = element_blank(),
    legend.position = "right",
    legend.spacing = unit(5, "pt"),
    legend.title = element_text(face = "bold", size = 9),
    legend.text = element_text(size = 8),
    legend.key.height = unit(12, "pt"),
    plot.margin = margin(10, 10, 10, 10)
  ) +
  guides(
    size = guide_legend(order = 1),
    color = guide_legend(order = 2),
    shape = guide_legend(order = 2)
  )
p_cloern
# Save as high-quality single-column figure
# ggsave(
#   filename = file.path(graph_dir, "FigX_Cloern_Limitations.png"),
#   plot = p_cloern,
#   device = "png",
#   width = 7,       # Standard single-column width (inches)
#   height = 5,        # Adjusted height for better proportions
#   units = "in",
#   dpi = 600,         # Publication quality
#   bg = "white"
# )

# For TIFF format (required by some journals):
#ggsave(filename = file.path(graph_dir, "Fig5_Cloern_Limitations.tiff"),  p_cloern,
#       width = 7, height = 5, units = "in", dpi = 600,
#       compression = "lzw")

In [ ]:
# 5. Phenology analysis
library(mgcv)
library(ggplot2)
library(lubridate)
library(plotly)
library(dplyr)

Jetty_df = read.csv(file=paste(wd_path,"/processed_data/","Jetty_HWseries.csv",sep=""), fileEncoding = "UTF-8", na.strings=c("", "NA"), header=TRUE)
Jetty_df$timestamp <- Jetty_df$timestamp%>%ymd_hms()
Jetty_df$doy <- yday(Jetty_df$timestamp)
Jetty_df$Day <- yday(Jetty_df$timestamp)
Jetty_df$Year <- year(Jetty_df$timestamp)
Jetty_df$DIN <- Jetty_df$NH4 + Jetty_df$NO2 + Jetty_df$NO3

plot(Jetty_df$timestamp,Jetty_df$PO4, ylab=c("PO4 (ug/L)"), t="b")
plot(Jetty_df$timestamp,Jetty_df$DIN, ylab=c("DIN (ug/L)"), t="b")
plot(Jetty_df$timestamp,Jetty_df$Si, ylab=c("DIN (ug/L)"), t="b")

Jetty_df$date <- as.Date(Jetty_df$timestamp)

Chla_1974_2012 = read.csv(file=paste(wd_path,"/rawdata/","Chla_CovertHPLC_1974_2012.csv",sep=""), fileEncoding = "UTF-8", na.strings=c("", "NA"), header=TRUE)
Chla_1974_2012$date_descr <- dmy(Chla_1974_2012$date_descr)

Jetty_df$Chl[which(Jetty_df$Year>=1990&Jetty_df$Year<=2011)] = Chla_1974_2012$HPLC_convmeas[which(year(Chla_1974_2012$date_descr)>=1990&year(Chla_1974_2012$date_descr)<=2011)] # replace Spectrophotometry-Chla with Converted HPLC-Chla (based on emperical relationship, HPLC-C ~ 0.83 * Spect-Chla)

# KNMI global radiation
hr_KNMI_filepath_1981_1990 <- paste(wd_path,"/rawdata/","uurgeg_235_1981-1990.txt",sep="")
hr_KNMI_filepath_1991_2000 <- paste(wd_path,"/rawdata/","uurgeg_235_1991-2000.txt",sep="")
hr_KNMI_filepath_2001_2010 <- paste(wd_path,"/rawdata/","uurgeg_235_2001-2010.txt",sep="")
hr_KNMI_filepath_2011_2020 <- paste(wd_path,"/rawdata/","uurgeg_235_2011-2020.txt",sep="")
hr_KNMI_filepath_2021_2030 <- paste(wd_path,"/rawdata/","uurgeg_235_2021-2030.txt",sep="")

hr_KNMI_deKooy_1981_1990 <- read.table(hr_KNMI_filepath_1981_1990, header=TRUE, sep=",")
hr_KNMI_deKooy_1991_2000 <- read.table(hr_KNMI_filepath_1991_2000, header=TRUE, sep=",")
hr_KNMI_deKooy_2001_2010 <- read.table(hr_KNMI_filepath_2001_2010, header=TRUE, sep=",")
hr_KNMI_deKooy_2011_2020 <- read.table(hr_KNMI_filepath_2011_2020, header=TRUE, sep=",")
hr_KNMI_deKooy_2021_2030 <- read.table(hr_KNMI_filepath_2021_2030, header=TRUE, sep=",")

hr_KNMI_deKooy_1981_1990$PAR = hr_KNMI_deKooy_1981_1990$Q*1e4/3600*2.515 # Conversion from KNMI (W m-2) to PAR (uE m-2 s-1), reference Brinkman et al., 2015, EMS-Dollard data report
hr_KNMI_deKooy_1991_2000$PAR = hr_KNMI_deKooy_1991_2000$Q*1e4/3600*2.515
hr_KNMI_deKooy_2001_2010$PAR = hr_KNMI_deKooy_2001_2010$Q*1e4/3600*2.515
hr_KNMI_deKooy_2011_2020$PAR = hr_KNMI_deKooy_2011_2020$Q*1e4/3600*2.515
hr_KNMI_deKooy_2021_2030$PAR = hr_KNMI_deKooy_2021_2030$Q*1e4/3600*2.515

hr_KNMI_deKooy_1981_1990$Datum <- paste(hr_KNMI_deKooy_1981_1990$YYYYMMDD, hr_KNMI_deKooy_1981_1990$HH,sep=" ")%>%ymd_h
hr_KNMI_deKooy_1991_2000$Datum <- paste(hr_KNMI_deKooy_1991_2000$YYYYMMDD, hr_KNMI_deKooy_1991_2000$HH,sep=" ")%>%ymd_h
hr_KNMI_deKooy_2001_2010$Datum <- paste(hr_KNMI_deKooy_2001_2010$YYYYMMDD, hr_KNMI_deKooy_2001_2010$HH,sep=" ")%>%ymd_h
hr_KNMI_deKooy_2011_2020$Datum <- paste(hr_KNMI_deKooy_2011_2020$YYYYMMDD, hr_KNMI_deKooy_2011_2020$HH,sep=" ")%>%ymd_h
hr_KNMI_deKooy_2021_2030$Datum <- paste(hr_KNMI_deKooy_2021_2030$YYYYMMDD, hr_KNMI_deKooy_2021_2030$HH,sep=" ")%>%ymd_h

hr_KNMI_deKooy <- rbind(hr_KNMI_deKooy_1981_1990, hr_KNMI_deKooy_1991_2000, hr_KNMI_deKooy_2001_2010, hr_KNMI_deKooy_2011_2020, hr_KNMI_deKooy_2021_2030)

PAR_frac = 0.45  # PAR is 45% of global radiation
Reflec_frac = 0.07 # reflecting of solar radiation at the surface by 5-10%, ref Brinkman et al., 2015
hr_KNMI_deKooy$PAR = hr_KNMI_deKooy$Q*PAR_frac*(1-Reflec_frac) # in (J/cm2/h) over an hour at water surface

# convert to photon flux (uE m-2 s-1); reference Brinkman et al., 2015,    EMS-Dollard data report
sec_per_hr = 3600 # seconds in an hour
cm2_to_m2 = 1e4 
conversion_factor = 4.66 # (μE/m2/s)/(W/m2); reference Jacobs et al., 2021, https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0246012 (in Brinkman et al., 2015 data report this conversion factor is 2.515)

day_ave_PAR <- aggregate(hr_KNMI_deKooy$PAR, by = list(as.Date(hr_KNMI_deKooy$Datum)),FUN=mean) # in (J/cm2/h)
names(day_ave_PAR) <- c("date","PAR")

day_ave_PAR$I_surf <- day_ave_PAR$PAR/3600*3600*24*cm2_to_m2*conversion_factor*1e-6 # in (mol E/m2/day)

plot(day_ave_PAR$date, day_ave_PAR$I_surf, t="l")

# Add a month column
day_ave_PAR <- day_ave_PAR[,c("date","I_surf")]
day_ave_PAR <- day_ave_PAR %>%
  mutate(year = year(date),
         month = month(date)
  )

Nut_Light_limitation_df = merge(Jetty_df, day_ave_PAR[,c("date","I_surf")], by.x = c("date"), by.y=c("date"))
Nut_Light_limitation_df$Day <- yday(Nut_Light_limitation_df$date)
Nut_Light_limitation_df$Year <- year(Nut_Light_limitation_df$date)

head(Nut_Light_limitation_df)
Nut_Light_limitation_df$I_w <- (Nut_Light_limitation_df$I_surf/(Nut_Light_limitation_df$Kd_Secci*4.6))*(1-exp(-(Nut_Light_limitation_df$Kd_Secci*4.6)))

plot(Nut_Light_limitation_df$date, Nut_Light_limitation_df$I_w)

Nut_Light_limitation_df$DIN <- Nut_Light_limitation_df$NO2 + Nut_Light_limitation_df$NO3 + Nut_Light_limitation_df$NH4

input_df <- Nut_Light_limitation_df[,c("date", "SampleID", "timestamp", "Day", "Year", "I_surf" ,"T","SecchiDepth","PO4","NO3","NO2", "NH4", "DIN", "Si", "Chl", "Kd_Secci", "I_w")]%>%na.omit

plot(input_df$date, input_df$I_w)

# range(gamm_input_df$NO3)

# based on the ratio Chl:C empirical relationship:
input_df$Carbon <- input_df$Chl/(0.003 + 0.0154*exp(0.050*input_df$T)*exp(-0.059*input_df$I_w)*(input_df$DIN/(input_df$DIN+1.5))) # KN taken from Cloern 1999

input_df[,8:ncol(input_df)] <- input_df[, 8:ncol(input_df)]%>%mutate(across(everything(), ~ifelse(. <=0, 0.001, .)))

input_df$DIN_avail = (input_df$DIN/(input_df$DIN+2)) # K_N = 2
input_df$Si_avail = (input_df$Si/(input_df$Si+2)) # K_N = 2
input_df$P_avail = (input_df$PO4/(input_df$PO4+0.2)) # K_P = 0.2

lim_Nut_vec = c("DIN", "DP", "DSi")
for (i in 1:nrow(input_df)){
  input_df$Nut_avail[i] = min(c(input_df$DIN_avail[i], input_df$P_avail[i], input_df$Si_avail[i]))
  input_df$limit_Nut[i] = lim_Nut_vec[which.min(c(input_df$DIN_avail[i], input_df$P_avail[i], input_df$Si_avail[i]))]
  
}

input_df$limit_Nut <- factor(input_df$limit_Nut, levels= c("DIN", "DP", "DSi"))

# gamm_input_df <- gamm_input_df[which(gamm_input_df$Year!=2008),]

# Function to fit the best model (two-dimensional smoother with AR-1 correlation structure)

fit_nlm_model <- function(response_var, input_data, family = gaussian()) {
  form <- as.formula(paste(response_var, "~ s(Day, bs = 'tp')"))
  nlm_model <- gam(form, 
                   family = family,
                   data = input_data)
  
  return(nlm_model)
}

fit_best_gam_model_year <- function(input_data, k_values = 6:10) {
  
  # Initialize a list to store models and AIC values
  models <- list()
  aic_values <- numeric(length(k_values))
  
  # Fit GAM models for each k value
  for (i in seq_along(k_values)) {
    k <- k_values[i]
    gam_model <- gam(
      Chl ~ te(Day, bs = 'cc', k = k),  # Cyclic spline for seasonal data enforce contiuity between year-end and start
      family = Gamma(link = "log"),
      method = "REML",  # Optimizes smoothness
      data = input_data
    )
    
    models[[i]] <- gam_model
    aic_values[i] <- AIC(gam_model)
  }
  
  # Find the best model (lowest AIC)
  best_idx <- which.min(aic_values)
  best_k <- k_values[best_idx]
  best_model <- models[[best_idx]]
  
  # Print AIC comparison
  aic_table <- data.frame(k = k_values, AIC = aic_values)
  print("year: ")
  print(input_data[1,])
  print("AIC Comparison:")
  print(aic_table)
  
  cat("\nBest model: k =", best_k, "(AIC =", aic_values[best_idx], ")\n")
  
  # Return the best model
  return(best_model)
}

# Create a grid of values for prediction
day_seq <- seq(1, 365, length.out = 365)

# Predict values using the fitted models for each year separately
pred_grid_list_model1 <- list()
# pred_grid_list_model2 <- list()


years <- unique(input_df$Year)
pheno_output <- data.frame(Year = years)

min_Chla <- min(input_df$Chl)

for (year in years) {
  
  # year = 1991
  pred_grid_year <- expand.grid(Day = day_seq, Year = year)
  pred_grid_year$date <- paste(pred_grid_year$Year,"-01-01")%>%ymd + pred_grid_year$Day-1
  
  meas_data <- input_df[which(input_df$Year==year), ]
  
  Chl_nlm <- fit_nlm_model(response_var = "Chl", input_data = meas_data,  family = Gamma(link = "log"))
  Iw_nlm <- fit_nlm_model(response_var = "I_w", input_data = meas_data,
family = gaussian())
  Tempw_nlm <- fit_nlm_model(response_var = "T", input_data = meas_data,
                             family = gaussian())
  availNut_nlm <- fit_nlm_model(response_var = "Nut_avail", input_data = meas_data,
                             family = gaussian())
  
  acf(resid(Chl_nlm)) # No spikes cross lines	Residuals are uncorrelated (independent).
  acf(resid(Iw_nlm)) # No spikes cross lines	Residuals are uncorrelated (independent).
  acf(resid(Tempw_nlm)) # No spikes cross lines	Residuals are uncorrelated (independent).
  
  AIC(Chl_nlm)
  summary(Chl_nlm)
  # gam.check(Chl_gam)
  
  # --- Predict with standard errors ---
  Chl_pred <- predict(Chl_nlm, newdata = pred_grid_year, se.fit = TRUE)
  Iw_pred <- predict(Iw_nlm, newdata = pred_grid_year, se.fit = TRUE)
  Tempw_pred <- predict(Tempw_nlm, newdata = pred_grid_year, se.fit = TRUE)
  availNut_pred <- predict(availNut_nlm, newdata = pred_grid_year, se.fit = TRUE)
  
  # --- For Chl, apply exp() because of log link (Gamma family) ---
  pred_grid_year[["Chl"]] <- exp(Chl_pred$fit)
  pred_grid_year[["Chl_upper"]] <- exp(Chl_pred$fit + 1.96 * Chl_pred$se.fit)
  pred_grid_year[["Chl_lower"]] <- exp(Chl_pred$fit - 1.96 * Chl_pred$se.fit)
  
  # --- For Gaussian models, no transformation needed ---
  pred_grid_year[["Iw"]] <- Iw_pred$fit
  pred_grid_year[["Iw_upper"]] <- Iw_pred$fit + 1.96 * Iw_pred$se.fit
  pred_grid_year[["Iw_lower"]] <- Iw_pred$fit - 1.96 * Iw_pred$se.fit
  
  pred_grid_year[["Tempw"]] <- Tempw_pred$fit
  pred_grid_year[["Tempw_upper"]] <- Tempw_pred$fit + 1.96 * Tempw_pred$se.fit
  pred_grid_year[["Tempw_lower"]] <- Tempw_pred$fit - 1.96 * Tempw_pred$se.fit
  
  pred_grid_year[["availNut"]] <- availNut_pred$fit
  pred_grid_year[["availNut_upper"]] <- availNut_pred$fit + 1.96 * availNut_pred$se.fit
  pred_grid_year[["availNut_lower"]] <- availNut_pred$fit - 1.96 * availNut_pred$se.fit
  
  pred_grid_year[["Chl_deriv"]] <- c(NA, diff(pred_grid_year[["Chl"]]))
  pred_grid_year[["Chl_der_perc"]] <- c(NA, diff(pred_grid_year[["Chl"]])/(pred_grid_year[["Chl"]][-length(pred_grid_year[["Chl"]])])*100)
  
  ###############################
  ######## Spring bloom #########
  ###############################
  t_Spr_peak_df = pred_grid_year[which(month(pred_grid_year$date)%in%3:5),]
  t_Spr_peak = t_Spr_peak_df$date[which.min(abs(t_Spr_peak_df$Chl_deriv))]
  pheno_output[which(pheno_output$Year==year),"t_Spr_peak"] = t_Spr_peak%>%yday
  
  Chl_t_Spr_peak = pred_grid_year$Chl[which(pred_grid_year$date==t_Spr_peak)]
  # Chl_t_Spr_peak = meas_data$Chl[which.min(abs(meas_data$date-t_Spr_peak))]
  
  
  pheno_output[which(pheno_output$Year==year),"Chl_t_Spr_peak"] = Chl_t_Spr_peak
  
  # slope of biomass growth
  pheno_output[which(pheno_output$Year==year),"Chl_spring_slope"] = Chl_t_Spr_peak/(yday(t_Spr_peak)) # mg/m3/day
  
  # underwater light summed over days:
  sum_light = sum(meas_data$I_w[which(meas_data$date <= t_Spr_peak)])
  pheno_output[which(pheno_output$Year==year),"sum_light"] = sum_light
  
  # winter DIN available for algae bloom
  DIN_stock = max(meas_data$DIN[which(meas_data$date <= t_Spr_peak)])
  pheno_output[which(pheno_output$Year==year),"DIN_stock"] = DIN_stock
  
  Remineralization_Spr_peak = max(meas_data$NH4[which(meas_data$date <= t_Spr_peak)] + meas_data$NO2[which(meas_data$date <= t_Spr_peak)])
  pheno_output[which(pheno_output$Year==year),"Remineralization_Spr_peak"] = Remineralization_Spr_peak
  
  # winter DP available for algae bloom
  DP_stock = max(meas_data$PO4[which(meas_data$date <= t_Spr_peak)])
  pheno_output[which(pheno_output$Year==year),"DP_stock"] = DP_stock
  
  # winter DSi available for algae bloom
  Si_stock = max(meas_data$Si[which(meas_data$date <= t_Spr_peak)])
  pheno_output[which(pheno_output$Year==year),"Si_stock"] = Si_stock
  
  # Winter nutrient availabibility
  availNut_Spr_peak = max(meas_data$Nut_avail[which(meas_data$date <= t_Spr_peak)])
  pheno_output[which(pheno_output$Year==year),"availNut_Spr_peak"] = availNut_Spr_peak
  
  ################################
  ######### Secondary bloom #########
  ################################
  t_2nd_peak_df = pred_grid_year[which(month(pred_grid_year$date)%in%6:11),]
  t_2nd_peak = t_2nd_peak_df$date[which.max(abs(t_2nd_peak_df$Chl))]
  
  Chl_2nd_peak = pred_grid_year$Chl[which(pred_grid_year$date==t_2nd_peak)]
  # Chl_t_Aut_peak = meas_data$Chl[which.min(abs(meas_data$date-t_Aut_peak))]
  pheno_output[which(pheno_output$Year==year),"t_2nd_peak"] = t_2nd_peak%>%yday
  pheno_output[which(pheno_output$Year==year),"Chl_2nd_peak"] = Chl_2nd_peak
  
  
  # water temperature (indicator of grazing pressure):
  maxWaterT = max(meas_data$T[which(meas_data$date>=t_Spr_peak & meas_data$date <= t_2nd_peak)])
  pheno_output[which(pheno_output$Year==year),"maxWaterT"] = maxWaterT
  
  # 2nd DIN stock
  DIN_stock_2ndpeak = max(meas_data$DIN[which(meas_data$date>=t_Spr_peak & meas_data$date <= t_2nd_peak)])
  pheno_output[which(pheno_output$Year==year),"DIN_stock_2ndpeak"] = DIN_stock_2ndpeak
  
  Remineralization_2ndpeak = max(meas_data$NH4[which(meas_data$date>=t_Spr_peak & meas_data$date <= t_2nd_peak)] + meas_data$NO2[which(meas_data$date>=t_Spr_peak & meas_data$date <= t_2nd_peak)])
  
  pheno_output[which(pheno_output$Year==year),"Remineralization_2ndpeak"] = Remineralization_2ndpeak
  
  # 2nd P stock
  P_stock_2ndpeak = max(meas_data$PO4[which(meas_data$date>=t_Spr_peak & meas_data$date <= t_2nd_peak)])
  pheno_output[which(pheno_output$Year==year),"P_stock_2ndpeak"] = P_stock_2ndpeak
  
  # 2nd Si stock
  Si_stock_2nd_peak = mean(meas_data$Si[which(meas_data$date>=t_Spr_peak & meas_data$date <= t_2nd_peak)])
  pheno_output[which(pheno_output$Year==year),"Si_stock_2nd_peak"] = Si_stock_2nd_peak
  
  # plot the seasonal patterns
  library(lubridate)
  
  # Open TIFF device
  
  tiff(filename = paste0(graph_dir,"/Phenology_graphs/",year, "_seasonal_patterns.tiff"), width=7, height=9, units="in", res=600, compression="lzw")
  # # 
  # Set up the plotting area with 3 rows and 1 column
 layout(matrix(1:3, ncol=1), heights=c(1, 1, 1))
 par(mar=c(0, 4, 0, 4), oma=c(4, 4, 4, 4))  # Shared outer margin

# --- Panel 1: I_w and Water Temperature ---
 plot(meas_data$Day, meas_data$I_w, type="p", col="#FFD700",
     ylab="", xaxt="n")
 mtext("Underwater Irradiance", side=2, line=2, col="#FFD700")

# Add confidence ribbon for Iw
 polygon(c(pred_grid_year$Day, rev(pred_grid_year$Day)),
        c(pred_grid_year$Iw_upper, rev(pred_grid_year$Iw_lower)),
        col=adjustcolor("#FFD700", alpha.f = 0.2), border=NA)

 lines(pred_grid_year$Day, pred_grid_year$Iw, type="l", col="#FFD700", xlab="", ylab="") 
 mtext(paste0(unique(pred_grid_year$Year), " Seasonal Patterns"), side =3, line = 1, col="black")

 par(new=TRUE)
 plot(meas_data$Day, meas_data$T, type="p", col="black", axes=FALSE, xlab="", ylab="", pch=2)
 axis(4, col.axis="black")
 mtext("Water Temp (°C)", side=4, line=2.5, col="black")

# Add confidence ribbon for temperature
 polygon(c(pred_grid_year$Day, rev(pred_grid_year$Day)),
        c(pred_grid_year$Tempw_upper, rev(pred_grid_year$Tempw_lower)),
        col=adjustcolor("black", alpha.f = 0.15), border=NA)

lines(pred_grid_year$Day, pred_grid_year$Tempw, type="l", col="black", xlab="", ylab="")

# --- Panel 2: Nutrient Availability ---
nut_colors <- c("DIN" = "#1F77B4", "DP" = "#9467BD", "DSi" = "#2CA02C")
nut_shapes <- c("DIN" = 16, "DP" = 17, "DSi" = 15)

plot(meas_data$Day, meas_data$Nut_avail, type="n", ylab="Nutrient Availability", xaxt="n")
points(meas_data$Day, meas_data$Nut_avail,
       col = nut_colors[meas_data$limit_Nut],
       pch = nut_shapes[meas_data$limit_Nut])

# Add confidence ribbon for availNut
polygon(c(pred_grid_year$Day, rev(pred_grid_year$Day)),
        c(pred_grid_year$availNut_upper, rev(pred_grid_year$availNut_lower)),
        col=adjustcolor("black", alpha.f = 0.15), border=NA)

lines(pred_grid_year$Day, pred_grid_year$availNut, type="l", col="black", xlab="", ylab="")

legend("bottomright", legend=names(nut_colors),
       col=nut_colors, pch=nut_shapes, pt.cex=1)

# --- Panel 3: Chl Derivative and Chl ---
plot(pred_grid_year$Day, pred_grid_year$Chl, type="l", col="green",
     xlab="", ylab="",)
points(meas_data$Day, meas_data$Chl, pch=16, cex=0.6)
axis(2, col.axis="green")
mtext("Chl (µg/L)", side=2, line= 2.5, col="green")
abline(v=ymd(c(t_Spr_peak, t_2nd_peak))%>%yday, lty=2)

# Add confidence ribbon for Chl
polygon(c(pred_grid_year$Day, rev(pred_grid_year$Day)),
        c(pred_grid_year$Chl_upper, rev(pred_grid_year$Chl_lower)),
        col=adjustcolor("green", alpha.f = 0.2), border=NA)

par(new=TRUE)
plot(pred_grid_year$Day, pred_grid_year$Chl_deriv, type="l", col="red",
     xlab="", ylab="", axes=F)
mtext("Chl Deriv", side= 4, line= 2.5, col="red")
axis(4, col.axis="red")
abline(h=0, lty=2)

# --- Shared x-axis label ---
mtext("Day of Year", side=1, outer=TRUE, line=2)

  
  dev.off()
  
}

# write.csv(pheno_output, file=paste(wd_path,"/processed_data/","pheno_output.csv",sep=""))

In [ ]:
## 5.1 *Phenology Figure*

library(ggplot2)
library(patchwork)
library(lubridate)
library(dplyr)

# Color-blind friendly colors
spring_col <- "#1b9e77"  # Green
fall_col <- "#d95f02"    # Orange
boxplot_fill <- "#f7f7f7"
boxplot_line <- "#7570b3"


Jetty_df <- read.csv(paste0(wd_path, "/processed_data/Jetty_HWseries.csv"),
                     fileEncoding = "UTF-8", na.strings = c("", "NA"), header = TRUE)

Jetty_df$timestamp <- ymd_hms(Jetty_df$timestamp)
Jetty_df$Year <- year(Jetty_df$timestamp)



Chla_1974_2012 = read.csv(file=paste(wd_path,"/rawdata/","Chla_CovertHPLC_1974_2012.csv",sep=""), fileEncoding = "UTF-8", na.strings=c("", "NA"), header=TRUE)
Chla_1974_2012$date_descr <- dmy(Chla_1974_2012$date_descr)

Jetty_df$Chl[which(Jetty_df$Year>=1990&Jetty_df$Year<=2011)] = Chla_1974_2012$HPLC_convmeas[which(year(Chla_1974_2012$date_descr)>=1990&year(Chla_1974_2012$date_descr)<=2011)] # replace Spectrophotometry-Chla with Converted HPLC-Chla (based on emperical relationship, HPLC-C ~ 0.83 * Spect-Chla)

Jetty_df_1990_2023 <- Jetty_df %>% filter(Year >= 1990 & Year <= 2023)


pheno_output <- read.csv(paste(wd_path,"/processed_data/","pheno_output.csv",sep=""))

# Panel A: Chlorophyll time series
panel_A <- ggplot(Jetty_df_1990_2023, aes(x = timestamp, y = Chl)) +
  geom_line(linewidth = 0.5, color = "#006400") +
  geom_point(size = 1, shape = 21, fill = "#006400", color = "black", stroke = 0.2) +
  labs(
    y = expression("Biomass (mg Chl-a m"^{-3}*")"),
    x = NULL,
    title = "A"
  ) +
  scale_x_datetime(date_breaks = "5 years", date_labels = "%Y") +
  theme_minimal() +
  theme(
    plot.title = element_text(size = 12, hjust = 0),
    axis.title = element_text(size = 10),
    axis.text = element_text(size = 9)
  )

# Panel B: Bloom Timing

panel_B <- ggplot(pheno_output, aes(x = Year, y = t_Spr_peak)) +
  geom_point(aes(x = Year, y = t_Spr_peak), size = 1.2, pch = 19, col = spring_col) +
  geom_smooth(aes(x = Year, y = t_Spr_peak), method = "loess", se = TRUE, alpha = 0.25, col = spring_col) +
  geom_point(aes(x = Year, y = t_2nd_peak), size = 1.2, pch = 17, col = fall_col) +
  geom_smooth(aes(x = Year, y = t_2nd_peak), method = "loess", se = TRUE, alpha = 0.25, col = fall_col) +
  labs(
    y = "Timing of bloom (day of year)",
    x = NULL,
    title = "B"
  ) +
  scale_color_manual(values = c("Spring bloom" = spring_col, "Secondary bloom" = fall_col)) +
  scale_shape_manual(values = c("Spring bloom" = 19, "Secondary bloom" = 17)) +
  theme_minimal() +
  theme(
    plot.title = element_text(size = 12, hjust = 0),
    axis.title = element_text(size = 10),
    axis.text = element_text(size = 9),
    legend.title = element_blank(),
    legend.position = "top"
  )

panel_B

# Panel C: Bloom Magnitude

library(tidyr)
library(dplyr)

# Long-format data
pheno_long <- pheno_output %>%
  pivot_longer(
    cols = c(Chl_Spr_peak, Chl_2nd_peak),
    names_to = "Bloom",
    values_to = "Chl"
  ) %>%
  mutate(Bloom = recode(Bloom,
                        "Chl_Spr_peak" = "Spring bloom",
                        "Chl_2nd_peak" = "Secondary bloom"))

panel_C <- ggplot(pheno_long, aes(x = Year, y = Chl, color = Bloom, shape = Bloom)) +
  geom_point(size = 1.2) +
  geom_smooth(method = "loess", se = TRUE, alpha = 0.25) +
  labs(
    y = expression("Bloom magnitude (mg Chl-a m"^{-3}*")"),
    x = "Year",
    title = "C"
  ) +
  scale_color_manual(
    values = c("Spring bloom" = spring_col, "Secondary bloom" = fall_col)
  ) +
  scale_shape_manual(
    values = c("Spring bloom" = 19, "Secondary bloom" = 17)
  ) +
  scale_y_continuous(position = "right") +
  theme_minimal() +
  theme(
    plot.title = element_text(size = 12, hjust = 0),
    axis.title = element_text(size = 10),
    axis.text = element_text(size = 9),
    legend.title = element_blank(),
    legend.justification = c(0, 1),
    legend.position = c(0.2, 1)
  )

panel_C


# Combine
combined_plot <- panel_A / (panel_B | panel_C) 
combined_plot






In [ ]:
## 5.2 Phenology statistic analysis
pheno_output <- read.csv(file=paste(wd_path,"/processed_data/","pheno_output.csv",sep=""))

cor(pheno_output%>%na.omit)

plot(pheno_output$t_Spr_peak, pheno_output$Nut_Dal_1)
plot(pheno_output$t_2nd_peak, pheno_output$Nut_Dal_2)
plot(pheno_output$t_2nd_peak, pheno_output$maxWaterT)

cor.test(pheno_output$t_Spr_peak, pheno_output$Nut_Dal_1, method = "pearson")
cor.test(pheno_output$Year, pheno_output$Chl_Spr_peak, method = "pearson")
cor.test(pheno_output$Chl_Spr_peak, pheno_output$DIN_stock, method = "pearson")
cor.test(pheno_output$Chl_Spr_peak, pheno_output$DP_stock, method = "pearson")
cor.test(pheno_output$Chl_Spr_peak, pheno_output$Si_stock, method = "pearson")

cor.test(pheno_output$t_2nd_peak, pheno_output$Nut_Dal_2, method = "pearson")
cor.test(pheno_output$t_2nd_peak, pheno_output$maxWaterT, method = "pearson")

cor.test(pheno_output$Chl_2nd_peak, pheno_output$maxWaterT, method = "pearson")
cor.test(pheno_output$Chl_2nd_peak, pheno_output$Year, method = "pearson")

cor.test(pheno_output$Chl_2nd_peak, pheno_output$Remineralization_2ndpeak, method = "pearson")
cor.test(pheno_output$Chl_2nd_peak, pheno_output$maxWaterT, method = "pearson")


cor.test(pheno_output$Year, pheno_output$Chl_Spr_peak, method = "pearson")

cor.test(pheno_output$Year, pheno_output$Chl_2nd_peak, method = "pearson")

# Create a proper binary indicator variable
pheno_output$post2010 <- ifelse(pheno_output$Year > 2010, "Y", "N")

# Convert to factor for proper analysis
pheno_output$post2010 <- as.factor(pheno_output$post2010)
# For more detailed comparison:
t.test(t_Spr_peak ~ post2010, data = pheno_output, var.equal = TRUE)

# first vs secondary bloom
t.test(pheno_output$Chl_Spr_peak, pheno_output$Chl_2nd_peak, var.equal = TRUE)


t.test(Chl_Spr_peak ~ post2010, data = pheno_output, var.equal = TRUE)
t.test(Chl_2nd_peak ~ post2010, data = pheno_output, var.equal = TRUE)


# Normality check
shapiro.test(pheno_output$t_2nd_peak[pheno_output$post2010 == "N"]) # If p-value > 0.05 → Fail to reject H₀ (data is normal).
shapiro.test(pheno_output$t_2nd_peak[pheno_output$post2010 == "Y"])

t.test(t_2nd_peak ~ post2010, data = pheno_output, var.equal = TRUE)
